<div class="alert alert-block alert-info"><b>ANN for running predictions on phycocyanin extraction data</b></div>

Relatively low amount of data points: 504 (used: 500 in 405 train/45 val/50 maneval)  
Inputs: 12 (currently)  
Outputs: 3

<b>To do:</b>
- DONE Rewrite with numpy arrays
- DONE Rewrite with Keras
- Rewrite with sklearn metrics
- Implement proper test run where appropriate
- DONE Rewrite for GPU support

<b>Notes:</b>
- The work directory path is: k:\Archive\Studying\NeuralNets\Python\Jupyter - Phycocyanin ANN\ - Don't forget to alter the work_path in the Extras file when copying!  
- For some reason only sigmoid seems to work
- 03/22: Keras fit() is causing a memory leak, using multiprocessing to bypass it. Importing matplotlib after running the model causes a kernel crash, using dump-restart-load to bypass it.
- Update 22/01/24: after switching to the GPU version of TensorFlow, the RAM leak appears to be gone - but VRAM usage cannot be cleared until the process is stopped
- tf.config.experimental.set_memory_growth(gpu, True) prevents TensorFlow from eating more than it needs, though
- matplotlib late import crash seems to be gone now

In [ ]:
# Designed as a general dependencies block that can be imported externally if necessary
import os
import psutil
import gc
import shutil
import time

import pandas as pd
import numpy as np
import tensorflow as tf
#from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.models import load_model
from tensorflow.keras.backend import clear_session

from itertools import cycle
from tensorflow.keras import backend as K # For the met_R2, if it even works
from tqdm.auto import tqdm
from tqdm.keras import TqdmCallback

#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"   # Disable GPU if necessary

In [ ]:
def noisy_duplicate_keras_b(data_array_loc): # alt version, might fix the memory issue
    data_array_new = data_array_loc.copy()
    for i in range(trainset_length): # Element-by-element
        for j in range(input_num_units):
            data_array_new[i][j] = data_array_new[i][j]*(1 + noise_factor*(0.5 - np.random.random_sample()))
    return data_array_new

def hp_config_print(hp_list):
    label_list = ["Config: ", "/", "/", " - ", "/", "/", ", B ", ", E/LR ", "/", ", EF/NF ", "/"]
    for item, label in zip(hp_list, label_list): print(label, item, sep='', end='')
    print()
    
def met_R2(y_true, y_pred):
    SS_res = K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res/(SS_tot + K.epsilon()))

def trainset_generator(x_array, y_array, batch_size): # Using a generator to feed the dataset
    pairs = [(x, y) for x in x_array for y in y_array]
    cycle_pairs = cycle(pairs)
    while (True):
        x_batch = []
        y_batch = []
        for _ in range(batch_size):
            x, y = next(cycle_pairs)
            x_batch.append(x)
            y_batch.append(y)
        yield np.array(x_batch), np.array(y_batch)

In [ ]:
# Non-multiprocess model maker and model caller in case the memory leak goes away
def keras_model_maker_nomp(work_path, hp_list, input_num_units, output_num_units):
    # Set RNG
    seed = 128
    rng = np.random.RandomState(seed)
    tf.random.set_seed(seed) # Perhaps I need to set tf. seed separately?
    # Keras: creating a model for the set of hyperparameters to be used for a single stat loop
    # The model is then saved and deleted from memory, no need to even return anything
    hidden1_num_units, hidden2_num_units, hidden3_num_units, hidden1_func, hidden2_func, hidden3_func, batch_size, epochs, learning_rate, extension_factor, noise_factor = hp_list
    model = Sequential([
        Dense(units=hidden1_num_units, input_shape=(input_num_units,), activation=hidden1_func),
        Dense(units=hidden2_num_units, activation=hidden2_func),
        Dense(units=hidden3_num_units, activation=hidden3_func),
        Dense(units=output_num_units, activation='linear')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=['mse'])
    try:
        shutil.rmtree(os.path.join(work_path, "ANN_temp"))
    except FileNotFoundError:
        pass    # Do nothing
    model.save(os.path.join(work_path, "ANN_temp", "model.keras"))
    del model
    clear_session()
    gc.collect()

def keras_model_fitter_nomp(work_path, train_in_array, train_out_array, batch_size, epochs, val_in_array):
    generator_enabled = False
    # Set RNG
    seed = 128
    rng = np.random.RandomState(seed)
    tf.random.set_seed(seed) # Perhaps I need to set tf. seed separately?
    # Loading a clear Keras model to train and evaluate
    model = load_model(os.path.join(work_path, "ANN_temp", "model.keras"))
    # Fit: suffle=False because it is already shuffled because of how the dataset is split between train and test
    if generator_enabled:
        trainset = trainset_generator(train_in_array, train_out_array, batch_size)
        model.fit(trainset, steps_per_epoch=len(train_in_array)/batch_size, batch_size=batch_size, epochs=epochs, shuffle=False, verbose=0, callbacks=[])
    else:
        model.fit(x=train_in_array, y=train_out_array, batch_size=batch_size, epochs=epochs, shuffle=False, verbose=0, callbacks=[])
    output_y = model(val_in_array, training=False)
    output_y = output_y.numpy() # Return as np.ndarray
    #print(type(output_y))
    del model
    clear_session()
    gc.collect()
    return output_y

In [ ]:
# Initializing variables and reading data
if __name__ == '__main__':  # Since this version uses multiprocessing
    print("Initialising...")
    work_path = os.path.join("k:" + os.sep, "Archive", "Studying", "NeuralNets", "Python", "Jupyter - Phycocyanin ANN")
    multiprocessing_enabled = False
    low_priority_enabled = False
    print("Multiprocessing is " + ("enabled" if multiprocessing_enabled else "disabled"))
    if multiprocessing_enabled:
        # Trying to sidestep the memory leak with multiprocessing
        import sys
        sys.path.append(work_path) # I really dislike this, but apparently there is no way to import it directly by path
        from Extras_v2 import keras_model_maker, keras_model_fitter # Importing Keras handling functions - for some reason required with Jupyter
        from multiprocessing import Process, Queue
    else:
        gpus = tf.config.experimental.list_physical_devices('GPU')
        for gpu in gpus: tf.config.experimental.set_memory_growth(gpu, True)
    print("Low process priority is " + ("enabled" if low_priority_enabled else "disabled"))
    # Set RNG
    seed = 128
    rng = np.random.RandomState(seed)
    tf.random.set_seed(seed) # Perhaps I need to set tf. seed separately?
    # Static parameters
    dataset_length = 504 # Everything
    trainset_length = 405 # Training
    valset_length = 45 # 10-fold cross-validation
    testset_length = 50 # Manual test set
    input_num_units = 12 #
    output_num_units = 3 # 3 is all
    start_time = time.time()
# End of __main__ section 

In [ ]:
# Reading and processing data: Pandas
if __name__ == '__main__':
    df = pd.read_csv(os.path.join(work_path, "Exp data SE 7 (good-504).csv"))
    df.info()
    print()
    df.drop(columns=['No', 'Init OD680', 'OD680'], inplace=True) # Drop the ID and 680s
    df = df.replace(to_replace='???', value=np.NaN) # Replace '???'s with true NaNs for ease of processing
    # Replace NaNs with means
    for i in df.columns[df.isnull().any(axis=0)]:
        df[i] = df[i].astype(np.float64) # Convert to float64 explicitly since there were string objects there
        df[i].fillna(df[i].mean(),inplace=True)
    mean_val = (df[df.columns.to_list()].mean()).to_numpy() # Not needed, testing only
    max_val = (df[df.columns.to_list()].max()) # Divide wont' work with a np_array of max values
    for i in df.columns:
        df[i] = df[i].div(max_val[i]) # Divide one by one because if not, the columns will get sorted alphabetically
    max_val = max_val.to_numpy()
    # Manually selected test dataset of 50 elements
    test_points_list = [ 22,  28,  32,  41,  51,  55,  68,  73,  85,  91,
                         96,  97, 109, 119, 124, 137, 150, 166, 173, 181,
                        188, 193, 204, 214, 222, 241, 250, 259, 267, 280,
                        292, 311, 319, 333, 342, 350, 361, 372, 378, 385,
                        390, 394, 401, 408, 415, 435, 450, 461, 473, 482]
    # Issues in lines 1 and 4
    # In 300, 311, 319, 333, 342
    # 300 causes decent improvement
    # 311 not much, 319 not much, 333 not much, 342 half-decent
    # Replacements for 300:
    # 290: meh 291: good 292: good+ 293: good+ 294: good+ 295: good 296: good+ 297: meh 298: good+ 299: good
    test_dataset = (df.loc[test_points_list]).to_numpy()
    df.drop(test_points_list, axis=0, inplace=True)
    # Now, make it into a numpy array
    dataset = df.to_numpy()
    del df
    #print("Max values are:\n", np.around(max_val, decimals=5))
    #print("Mean values are:\n", np.around(mean_val, decimals=5))
    #print("Processed dataset example:\n", dataset[55])
    print("Dataset shape is:", dataset.shape)
# End of __main__ section

In [ ]:
# Setting hyperloop and statistical vars
if __name__ == '__main__':
    # For best results (these are light enough to keep in lists, and I want my Fs):
    MSE_best_val = ['F']*output_num_units
    MSE_best_id = [0]*output_num_units
    MSE_best_config = [0]*output_num_units  # Will be a list of lists
    R2_best_val = ['F']*output_num_units
    R2_best_id = [0]*output_num_units
    R2_best_config = [0]*output_num_units
    stat_block_MSE_big = np.empty((0, output_num_units))
    stat_block_R2_big = np.empty((0, output_num_units))
    # Hyperparameters
    # Article hp_config: [10, 8, 9, 'sigmoid', 'sigmoid', 'sigmoid', 10, 5000, 0.001, 4, 0.05]
    hyperloop_type = "grid" # This one line should define which hyperloop type we're using
    hyperparameters_grid = [
        [5, 7, 10, 15, 25],          # Layer 1 neurons
        [5, 7, 10, 15, 25],          # Layer 2 neurons
        [5, 7, 10, 15, 25],          # Layer 3 neurons
        ['sigmoid', 'tanh', 'relu'],    # Layer 1 acti function
        ['sigmoid', 'tanh', 'relu'],    # Layer 2 acti function
        ['sigmoid', 'tanh', 'relu'],    # Layer 3 acti function
        [10, 40, 100],                    # Batch size
        [500, 1500, 5000],          # Epochs num
        [0.001, 0.005, 0.025, 0.1],    # Learning rate
        [0, 1, 4],               # Extension factor
        [0, 0.01, 0.05]               # Noise amount
        ]
    hyperparameters_list = [
        [10, 8, 9, 'tanh', 'tanh', 'tanh', 10, 2500, 0.01, 0, 0.05],
        [10, 8, 9, 'tanh', 'tanh', 'tanh', 10, 1500, 0.01, 1, 0.05],
        [10, 8, 9, 'tanh', 'tanh', 'tanh', 10, 1000, 0.01, 4, 0.05]
        ]
    hyperparameters_sing = [5, 7, 7, 'sigmoid', 'sigmoid', 'sigmoid', 50, 2500, 0.01, 0, 0.0]
    hyperparameters_grid_start = [10, 10, 10, 'sigmoid', 'sigmoid', 'sigmoid', 40, 1500, 0.025, 0, 0.01]
    hyperparameters_rand_start = [0]*len(hyperparameters_grid)
    #
    if hyperloop_type == "sing":
        hyperloop_count = 1
    elif hyperloop_type == "list":
        hyperloop_count = len(hyperparameters_list)
    elif hyperloop_type == "grid":
        passes_count = 2
        hyperloop_count = 0
        for hl_line in hyperparameters_grid:
            hyperloop_count += len(hl_line)
        hyperloop_count = hyperloop_count*passes_count + 1
    elif hyperloop_type == "rand":  # Hyperparameters are chosen by The Dragon Reborn!
        hyperloop_count = 10    # Manual
    #
    val_fold_count = 10     # 10
    # Debug: tracing memory
    this_process = psutil.Process(os.getpid())
    if low_priority_enabled: this_process.nice(psutil.BELOW_NORMAL_PRIORITY_CLASS)
    print("Initial memory usage:", "{:.1f}".format(this_process.memory_info().rss/1048576), "MB")
# End of __main__ section

In [ ]:
# The hyperloop itself
if __name__ == '__main__':
    print("Starting hyperloop...")
    hyperloop_i = 0
    while hyperloop_i < hyperloop_count:    # Using while because I need to be able to make adjustments within the loop
        # Getting hyperparameters
        if hyperloop_type == "sing":
            hyperparameters = hyperparameters_sing
        elif hyperloop_type == "list":
            hyperparameters = hyperparameters_list[hyperloop_i]
        #
        elif hyperloop_type == "grid":
            if hyperloop_i == 0:    # Grid requires an init, I'll keep it here
                hyperparameters = hyperparameters_grid_start
                hpg_i = 0   # Parameter type
                hpg_j = 0   # Parameter value
                hpg_k = 0   # Passes done
                badness = []  # Quality of a choice within a parameter list
            elif hyperloop_i == 999:
                break
            #
            elif hpg_j == len(hyperparameters_grid[hpg_i]) - 1:   # Went through one parameter?
                hyperparameters[hpg_i] = hyperparameters_grid[hpg_i][badness.index(min(badness))]   # Set this param to the value with minimal badness
                print(">> Parameter no. ", hpg_i+1, ', best option: ', str(hyperparameters_grid[hpg_i][badness.index(min(badness))]), sep='')
                hpg_j = 0
                hpg_i = hpg_i + 1   # Switch to the next parameter
                if hpg_i != len(hyperparameters_grid):
                    while len(hyperparameters_grid[hpg_i]) == 1: # Is this a pointless single-value parameter? Skip
                        print(">> Parameter no. ", hpg_i+1, ': single option, skipping', sep='')
                        hpg_i = hpg_i + 1
                        hyperloop_i = hyperloop_i + 1
                badness = []
                if hpg_i == len(hyperparameters_grid):      # Went through the list once? Start again
                    hpg_i = 0   # Can only trigger if we went through a parameter anyway
                    hpg_k = hpg_k + 1
                    print(">> Passes comlpete: ", hpg_k, '/', passes_count, sep='')
            else:   # No? Keep going
                hpg_j = hpg_j + 1
            #
            if hpg_k == passes_count:   # Done? Make one more cycle with the supposed optimal config
                #hyperloop_i = 999
                print(">> Optimization complete, testing configuration...")
            else:
                hyperparameters[hpg_i] = hyperparameters_grid[hpg_i][hpg_j]
        #
        elif hyperloop_type == "rand":
            hyperparameters = [0]*len(hyperparameters_grid)
            for hpr_i in range(len(hyperparameters_grid)):
                hyperparameters[hpr_i] = np.random.choice(hyperparameters_grid[hpr_i]) # Just pick a random option for every hyperparameter
        # Hyperparameters set, preparing the statloop
        print("\n- ", hyperloop_i+1, "/", hyperloop_count, " -", sep='')
        hidden1_num_units, hidden2_num_units, hidden3_num_units, hidden1_func, hidden2_func, hidden3_func, batch_size, epochs, learning_rate, extension_factor, noise_factor = hyperparameters
        # Data normalization for the noise
        stat_block_MSE = np.zeros((val_fold_count, output_num_units))    # Big block now
        stat_block_R2 = np.zeros((val_fold_count, output_num_units))

        if multiprocessing_enabled:
            # Everything actually iteratively done with Keras should be in a separate process
            # Keras: creating a model for the set of hyperparameters to be used for a single stat loop
            #print("Trying to make a model...", end=' ')
            model_q = Queue()
            model_p = Process(target=keras_model_maker, args=(model_q, work_path, hyperparameters, input_num_units, output_num_units))
            model_p.start()
            ans = model_q.get() # So we can be sure it completed?..
            model_p.terminate()
            #print("Model is made!")
        else: # Just call the non-multiprocessing version of the function
            keras_model_maker_nomp(work_path, hyperparameters, input_num_units, output_num_units)
        
        hp_config_print(hyperparameters)
        for val_loop_i in tqdm(range(val_fold_count)):
            # Pick the validation fold out first, then shuffle and inflate
            train_in_array = np.zeros((trainset_length, input_num_units))
            train_out_array = np.zeros((trainset_length, output_num_units))
            val_in_array = np.zeros((valset_length, input_num_units))
            val_out_array = np.zeros((valset_length, output_num_units))
            # Using method D: random without repeats
            train_dataset = np.empty((0, input_num_units+output_num_units))
            val_dataset = np.empty((0, input_num_units+output_num_units))
            val_start = valset_length*val_loop_i
            val_end = valset_length*(val_loop_i + 1)
            #print(" val_loop_i, val_start, val_end are:", val_loop_i, val_start, val_end)
            if val_loop_i == 0:
                rand_list = np.random.choice(trainset_length + valset_length, trainset_length + valset_length, False)
                rand_list_special = rand_list.copy()
            for i in range(trainset_length + valset_length):
                if i in rand_list_special[val_start:val_end]:
                    val_dataset = np.concatenate(( val_dataset, np.array( [dataset[i].copy()] ) ))
                else:
                    train_dataset = np.concatenate(( train_dataset, np.array( [dataset[i].copy()] ) ))
            #print("train_dataset shape is:", train_dataset.shape)
            #print("val_dataset shape is:", val_dataset.shape)
            rand_list = np.random.choice(trainset_length, trainset_length, False)
            for i in range(trainset_length):
                line_to_add = train_dataset[rand_list[i]]
                train_in_array[i] = (line_to_add[:input_num_units].copy())/(1 + noise_factor)
                train_out_array[i] = (line_to_add[input_num_units:].copy())
            for i in range(valset_length):
                line_to_add = val_dataset[i]
                val_in_array[i] = (line_to_add[:input_num_units].copy())/(1 + noise_factor)
                val_out_array[i] = (line_to_add[input_num_units:].copy())
            # Extending dataset with 'noisy' duplicates
            train_in_array_temp = train_in_array.copy()
            train_out_array_temp = train_out_array.copy()
            for i in range(0, extension_factor):
                train_in_array_temp = np.concatenate((train_in_array_temp, noisy_duplicate_keras_b(train_in_array)))  # Noisy inputs
                train_out_array_temp = np.concatenate((train_out_array_temp, train_out_array))                      # Clean corresponding outputs
            train_in_array = train_in_array_temp.copy()    # Copy because I'm clearing it, which is probably unnecessary
            train_out_array = train_out_array_temp.copy()
            del train_in_array_temp
            del train_out_array_temp
            #print("Train input shape is:", train_in_array.shape, "train output shape is:", train_out_array.shape)
            #print("Val input shape is:", val_in_array.shape, "val output shape is:", val_out_array.shape)

            if multiprocessing_enabled:
                # Everything actually iteratively done with Keras should be in a separate process
                #print("Trying to call the fitting function", end='')
                fit_q = Queue()
                fit_p = Process(target=keras_model_fitter, args=(fit_q, work_path, train_in_array, train_out_array, batch_size, epochs, val_in_array))
                fit_p.start()
                output_y = fit_q.get()
                fit_p.terminate()
                #print("output_y is:", type(output_y), output_y.shape)
            else: # Just call the non-multiprocessing version of the function
                output_y = keras_model_fitter_nomp(work_path, train_in_array, train_out_array, batch_size, epochs, val_in_array)
    
            # Manual statistics - rewritten
            y_eta = np.transpose(val_out_array) # (Array of) arrays of values for each output
            y_pred = np.transpose(output_y)
            for i in range(output_num_units):
                stat_block_MSE[val_loop_i][i] = np.mean(np.square(np.subtract(np.asarray(y_eta[i]), np.asarray(y_pred[i]))))
            for i in range(output_num_units):
                RSS = np.sum(np.square(np.subtract(np.asarray(y_eta[i]), np.asarray(y_pred[i]))))
                TSS = np.sum(np.square(np.subtract(np.asarray(y_eta[i]), np.mean(np.asarray(y_eta[i])))))
                stat_block_R2[val_loop_i][i] = np.subtract(1.0, np.divide(RSS, TSS))
            del train_dataset
            del val_dataset
            del train_in_array
            del train_out_array
            del val_in_array
            del val_out_array
            del output_y
            del y_eta
            del y_pred
            # End of cross-validation loop
        #print("\nFull stat block: MSE:\n", stat_block_MSE)
        #print("\nFull stat block: R2:\n", stat_block_R2)
        stat_block_MSE = np.median(stat_block_MSE, axis=0)
        stat_block_R2 = np.median(stat_block_R2, axis=0)
        print("Statistics:")
        print("MSE: ", end='')
        print(np.around(stat_block_MSE, decimals=5))
        print("R2: ", end='')
        print(np.around(stat_block_R2, decimals=5))
        # Remember best results
        if hyperloop_type == 'list':
            for i in range(output_num_units):
                if MSE_best_val[i] == 'F' or MSE_best_val[i] > stat_block_MSE[i]:
                    MSE_best_val[i] = stat_block_MSE[i]
                    MSE_best_id[i] = hyperloop_i + 1
                if R2_best_val[i] == 'F' or R2_best_val[i] < stat_block_R2[i]:
                    R2_best_val[i] = stat_block_R2[i]
                    R2_best_id[i] = hyperloop_i + 1
        elif hyperloop_type == 'grid':
            #badness.append(np.sum(stat_block_MSE))  # Optimising for MSE or R2? Sum is very basic, but will do for now
            badness.append(-np.sum(stat_block_R2))   # Flipping the R2 sum so we can always look lowest badness
        elif hyperloop_type == 'rand':
            for i in range(output_num_units):
                if MSE_best_val[i] == 'F' or MSE_best_val[i] > stat_block_MSE[i]:
                    MSE_best_val[i] = stat_block_MSE[i]
                    MSE_best_config[i] = hyperparameters.copy() # Saving configs instead of IDs
                if R2_best_val[i] == 'F' or R2_best_val[i] < stat_block_R2[i]:
                    R2_best_val[i] = stat_block_R2[i]
                    R2_best_config[i] = hyperparameters.copy()
        # Went through a set of parameters => clear the model
        #if hyperloop_i != hyperloop_count - 1:
        #    del model
        #    keras.backend.clear_session()
        #    tf.compat.v1.reset_default_graph()
        stat_block_MSE_big = np.concatenate( (stat_block_MSE_big, np.array( [stat_block_MSE.copy()] )) )
        stat_block_R2_big = np.concatenate( (stat_block_R2_big, np.array( [stat_block_R2.copy()] )) )
        del stat_block_MSE
        del stat_block_R2
        shutil.rmtree(os.path.join(work_path, "ANN_temp"))
        #model.summary() # Debug
        hyperloop_i = hyperloop_i + 1
        print("Memory usage:", "{:.1f}".format(this_process.memory_info().rss/1048576), "MB")
        # End of hyperparameter loop
    print("\nDone! Total work time: %s seconds" % (time.time() - start_time))
# End of __main__ section

In [ ]:
# Now printing (and saving/plotting) stats
if __name__ == '__main__':
    if hyperloop_type == 'list':
        print("Best results:")
        print("Best MSE:", end='')
        print(np.around(MSE_best_val, decimals=5))
        print("Achieved:", end='')
        print(MSE_best_id)
        print("Best R2:", end='')
        print(np.around(R2_best_val, decimals=5))
        print("Achieved:", end='')
        print(R2_best_id)
    #elif hyperloop_type == 'grid':
        # Nothing needs doing
    elif hyperloop_type == 'rand':
        print("Best results:")
        print("Best MSE:", end='')
        print(np.around(MSE_best_val, decimals=5))
        print("Achieved:")
        for hpc_line in MSE_best_config: hp_config_print(hpc_line)
        print("Best R2:", end='')
        print(np.around(R2_best_val, decimals=5))
        print("Achieved:")
        for hpc_line in R2_best_config: hp_config_print(hpc_line)
    print("Print big stat block? [y]/n")
    ans = input('')
    if ans != 'n':
        print("'': {")
        for i in range(output_num_units):
            print(("    'MSE_OD': ", "    'MSE_pH': ", "    'MSE_mrel': ")[i], end='')
            for j in range(len(stat_block_MSE_big)):
                print('[' if j == 0 else ', ', np.around(stat_block_MSE_big[j][i], decimals=5), sep='', end='')
            print('],')
        for i in range(output_num_units):
            print(("    'R2_OD': ", "    'R2_pH': ", "    'R2_mrel': ")[i], end='')
            for j in range(len(stat_block_R2_big)):
                print('[' if j == 0 else ', ', np.around(stat_block_R2_big[j][i], decimals=5), sep='', end='')
            print('],')
        print("    'x_arr':\n}")
# End of __main__ section

<div class="alert alert-block alert-info"><b>Plotters for the output (and cells to load existing stat data into them directly)</b></div>

In [ ]:
# Loader for 'List' stat blocks
if __name__ == '__main__':
    print("Loading \"List\" test stat block...", end = '')
    stat_blocks = {
        # 1
        '1': {
            'MSE_OD': [0.00591, 0.00527, 0.00405, 0.00346, 0.00385, 0.00489, 0.00454, 0.00343, 0.00366, 0.0033, 0.00313, 0.00407, 0.00392, 0.00425, 0.0049, 0.00391, 0.00397, 0.00415, 0.00361, 0.00301],
            'MSE_pH': [0.00063, 0.00049, 0.00068, 0.00056, 0.0004, 0.00053, 0.00052, 0.00041, 0.00035, 0.00049, 0.00035, 0.00038, 0.00043, 0.00041, 0.00039, 0.00036, 0.00037, 0.00048, 0.00032, 0.00045],
            'MSE_mrel': [0.01221, 0.01484, 0.01586, 0.01186, 0.0112, 0.01315, 0.00901, 0.01401, 0.00913, 0.00889, 0.01151, 0.01183, 0.00685, 0.0098, 0.00945, 0.01539, 0.00959, 0.00837, 0.01058, 0.01369],
            'R2_OD': [0.8404, 0.84908, 0.89116, 0.92329, 0.9124, 0.8892, 0.88138, 0.91691, 0.92712, 0.92314, 0.92127, 0.88913, 0.89394, 0.89677, 0.86124, 0.91522, 0.90422, 0.855, 0.90894, 0.93359],
            'R2_pH': [0.70039, 0.70299, 0.70696, 0.72405, 0.81047, 0.73337, 0.74643, 0.80213, 0.84712, 0.7519, 0.82786, 0.80478, 0.79733, 0.81271, 0.83193, 0.8277, 0.84647, 0.79834, 0.84156, 0.8086],
            'R2_mrel': [0.71876, 0.76571, 0.73156, 0.81275, 0.77976, 0.78793, 0.84032, 0.73858, 0.81286, 0.8482, 0.80456, 0.78937, 0.87749, 0.7904, 0.82773, 0.76006, 0.80864, 0.82073, 0.79806, 0.7371],
            'x_arr': list(range(1, 21))
        },
        # 2: 19.04.22, Config: 15, 15, 15, 'sigmoid', 'sigmoid', 'sigmoid', 50, 5000, 0.01, 0, ???
        '2': {
            'MSE_OD': [0.00417, 0.00361, 0.0033, 0.00564, 0.00208, 0.003, 0.00122, 0.00339, 0.00132, 0.00266, 0.00328, 0.00297, 0.00381, 0.00223, 0.00304, 0.00388, 0.00306, 0.00133, 0.00269, 0.00319],
            'MSE_pH': [0.0004, 0.00028, 0.00026, 0.0002, 0.00032, 0.00025, 0.00023, 0.00023, 0.00019, 0.00022, 0.00028, 0.00019, 0.00021, 0.00033, 0.00018, 0.00038, 0.00023, 0.00012, 0.00016, 0.00016],
            'MSE_mrel': [0.01141, 0.01118, 0.00904, 0.01065, 0.01071, 0.01239, 0.00503, 0.01271, 0.00904, 0.00567, 0.00758, 0.00653, 0.00727, 0.00729, 0.00943, 0.01419, 0.00896, 0.00434, 0.006, 0.00845],
            'R2_OD': [0.88447, 0.90245, 0.9184, 0.85283, 0.94342, 0.93028, 0.97169, 0.89506, 0.96479, 0.93116, 0.92164, 0.93062, 0.89874, 0.95161, 0.93474, 0.8985, 0.94605, 0.96558, 0.9384, 0.91682],
            'R2_pH': [0.8514, 0.87793, 0.85922, 0.87997, 0.87102, 0.90279, 0.89561, 0.87354, 0.90828, 0.91793, 0.89112, 0.9287, 0.91575, 0.83879, 0.87406, 0.80071, 0.89353, 0.93624, 0.93133, 0.92768],
            'R2_mrel': [0.77427, 0.79109, 0.83392, 0.75781, 0.80102, 0.75568, 0.88548, 0.71741, 0.84435, 0.89435, 0.87239, 0.8552, 0.88435, 0.88388, 0.83291, 0.72468, 0.82259, 0.91847, 0.86645, 0.85533],
            'x_arr': list(range(1, 21))
        },
        # 3: 02.11.22, Config: 7/7/7 - sigmoid/sigmoid/sigmoid, B 50, E/LR 5000/0.01, EF/NF 4/0.0~0.27
        '3': {
            'MSE_OD': [0.00367, 0.00394, 0.00428, 0.0045, 0.00381, 0.00429, 0.00524, 0.00429, 0.0047, 0.00518],
            'MSE_pH': [0.00074, 0.00079, 0.00088, 0.00078, 0.0009, 0.00105, 0.00093, 0.00081, 0.00096, 0.00082],
            'MSE_mrel': [0.00945, 0.0105, 0.01025, 0.0099, 0.01171, 0.01002, 0.01005, 0.01299, 0.01108, 0.01211],
            'R2_OD': [0.91995, 0.91386, 0.91804, 0.90697, 0.91229, 0.90838, 0.87585, 0.91566, 0.90536, 0.89501],
            'R2_pH': [0.81039, 0.80145, 0.79328, 0.80055, 0.77714, 0.74062, 0.75696, 0.81522, 0.76547, 0.78068],
            'R2_mrel': [0.78595, 0.74458, 0.75207, 0.77257, 0.69652, 0.73562, 0.74698, 0.6823, 0.74915, 0.71838],
            'x_arr': [x*0.03 for x in range(0, 10)]
        },
        # 4: 07.12.22, Config: 7/7/7 - sigmoid/sigmoid/sigmoid, B 50, E/LR 500~5000/0.01, EF/NF 4/0.15
        '4': {
            'MSE_OD': [0.00614, 0.00566, 0.00407, 0.00436, 0.005, 0.00423, 0.00452, 0.00497, 0.00401, 0.00395],
            'MSE_pH': [0.00097, 0.0009, 0.00093, 0.00085, 0.00068, 0.00083, 0.00076, 0.00085, 0.00087, 0.00109],
            'MSE_mrel': [0.01577, 0.01212, 0.01343, 0.01093, 0.01103, 0.01143, 0.01215, 0.01227, 0.01134, 0.01097],
            'R2_OD': [0.88378, 0.89707, 0.91923, 0.90565, 0.90426, 0.92254, 0.89958, 0.89947, 0.923, 0.92049],
            'R2_pH': [0.75573, 0.76127, 0.78651, 0.80711, 0.83105, 0.80988, 0.80819, 0.79756, 0.81252, 0.80304],
            'R2_mrel': [0.6081, 0.7229, 0.71451, 0.73521, 0.72337, 0.77269, 0.72232, 0.70512, 0.75538, 0.7377],
            'x_arr': [x*500 for x in range(1, 11)]
        },
        # 5: 08.12.22, Config: 7/7/7 - sigmoid/sigmoid/sigmoid, B 50, E/LR 500~5000/0.01, EF/NF 4/0.15
        '5': {
            'MSE_OD': [0.00548, 0.00464, 0.00542, 0.00416, 0.00439, 0.00453, 0.00463, 0.00494, 0.00442, 0.00483],
            'MSE_pH': [0.00081, 0.00084, 0.00083, 0.00095, 0.00076, 0.00091, 0.00087, 0.00107, 0.00093, 0.00102],
            'MSE_mrel': [0.01286, 0.01307, 0.01083, 0.01193, 0.00976, 0.01124, 0.0114, 0.01166, 0.01194, 0.00935],
            'R2_OD': [0.88606, 0.90777, 0.88839, 0.9196, 0.91535, 0.90887, 0.91952, 0.90782, 0.89782, 0.90987],
            'R2_pH': [0.80329, 0.79843, 0.78745, 0.79197, 0.83218, 0.76612, 0.79335, 0.77106, 0.75837, 0.77191],
            'R2_mrel': [0.65466, 0.72412, 0.71629, 0.75581, 0.78141, 0.72786, 0.72401, 0.73632, 0.73784, 0.77338],
            'x_arr': [x*500 for x in range(1, 11)]
        },
        # 6: 18.01.24, Config: 10/8/9 - sigmoid/sigmoid/sigmoid, B 10, E/LR 2500/0.01, EF/NF [0, 1, 2, 3, 4, 5, 7, 9, 11]/0.05
        '6': {
            'MSE_OD': [0.00477, 0.00456, 0.00565, 0.00341, 0.00292, 0.00547, 0.0035, 0.00267, 0.00393],
            'MSE_pH': [0.00102, 0.00095, 0.00075, 0.00095, 0.00099, 0.00094, 0.00073, 0.00097, 0.00085],
            'MSE_mrel': [0.01155, 0.00872, 0.00972, 0.01061, 0.00892, 0.01264, 0.01041, 0.00989, 0.00727],
            'R2_OD': [0.90294, 0.91247, 0.89001, 0.93451, 0.93869, 0.89214, 0.93223, 0.93366, 0.93077],
            'R2_pH': [0.75105, 0.81611, 0.81827, 0.80051, 0.76628, 0.78762, 0.8573, 0.80018, 0.8157],
            'R2_mrel': [0.71577, 0.78406, 0.75714, 0.72675, 0.75637, 0.69899, 0.76055, 0.74843, 0.83475],
            'x_arr': [0, 1, 2, 3, 4, 5, 7, 9, 11]
        },
        # 7: 19.01.24, Config: 10/8/9 - sigmoid/sigmoid/sigmoid, B 10, E/LR 2500/0.01, EF/NF [0, 1, 2, 3, 4, 5, 7, 9, 11, 15, 19]/0.05
        '7': {
            'MSE_OD': [0.00453, 0.00469, 0.00488, 0.00389, 0.00507, 0.00564, 0.00392, 0.00397, 0.00341, 0.00463, 0.00442],
            'MSE_pH': [0.00119, 0.00089, 0.00083, 0.00101, 0.00098, 0.00084, 0.00081, 0.00073, 0.00068, 0.00109, 0.00079],
            'MSE_mrel': [0.01013, 0.00995, 0.00979, 0.00845, 0.00856, 0.0107, 0.00831, 0.01028, 0.00707, 0.0105, 0.01081],
            'R2_OD': [0.90986, 0.91302, 0.89601, 0.92303, 0.89639, 0.88923, 0.91802, 0.9209, 0.92648, 0.90763, 0.91779],
            'R2_pH': [0.75988, 0.80903, 0.81938, 0.79518, 0.80988, 0.80527, 0.83632, 0.83839, 0.84517, 0.83553, 0.83125],
            'R2_mrel': [0.75761, 0.73778, 0.74381, 0.77073, 0.76316, 0.72877, 0.79968, 0.74751, 0.83691, 0.75646, 0.75295],
            'x_arr': [0, 1, 2, 3, 4, 5, 7, 9, 11, 15, 19]
        },
        # 7b: 21.01.24, Config: 10/8/9 - sigmoid/sigmoid/sigmoid, B 10, E/LR [1250, 1250, 1000, 1000]/0.01, EF/NF [10, 12, 13, 14]/0.05
        '7b': {
            'MSE_OD': [0.00405, 0.00465, 0.00372, 0.00456],
            'MSE_pH': [0.00078, 0.00088, 0.00087, 0.00105],
            'MSE_mrel': [0.00794, 0.00833, 0.01059, 0.01021],
            'R2_OD': [0.92679, 0.9184, 0.9295, 0.91529],
            'R2_pH': [0.84471, 0.80584, 0.8034, 0.77537],
            'R2_mrel': [0.8114, 0.79367, 0.77873, 0.74838]
        },
        # A copy of 7, edited for a nice picture !! SWITCH 6th point to 2nd place
        'e': {
            'MSE_OD': [0.00453, 0.00389, 0.00392, 0.0035, 0.00341, 0.00507, 0.00397, 0.00442, 0.00488, 0.00469, 0.00564],
            'MSE_pH': [0.00119, 0.00101, 0.00081, 0.00073, 0.00068, 0.00098, 0.00073, 0.00079, 0.00083, 0.00089, 0.00084],
            'MSE_mrel': [0.01013, 0.00845, 0.00831, 0.00941, 0.00707, 0.00856, 0.01028, 0.00981, 0.00979, 0.00995, 0.0107],
            'R2_OD': [0.90986, 0.92303, 0.91802, 0.93223, 0.92648, 0.89639, 0.9209, 0.91779, 0.89601, 0.91302, 0.88923],
            'R2_pH': [0.75988, 0.79518, 0.83632, 0.8573, 0.84517, 0.80988, 0.83839, 0.83125, 0.81938, 0.80903, 0.80527],
            'R2_mrel': [0.75761, 0.77073, 0.79968, 0.80055, 0.83691, 0.76316, 0.74751, 0.75295, 0.74381, 0.73778, 0.72877],
            'x_arr': [0, 1, 2, 3, 4, 5, 7, 9, 11, 15, 19]
        },
        # 8: 23.01.24, Config: 10/8/9 - relu/relu/relu, B 10, E/LR (2500, 1500, 1000)/0.01, EF/NF (0, 1, 4)/0.05, generator is disabled
        '8': {
            'MSE_OD': [0.01084, 0.00999, 0.00919],
            'MSE_pH': [0.00238, 0.00218, 0.00239],
            'MSE_mrel': [0.01879, 0.01728, 0.01333],
            'R2_OD': [0.80002, 0.82738, 0.80415],
            'R2_pH': [0.50739, 0.52167, 0.50365],
            'R2_mrel': [0.49387, 0.61101, 0.68568],
            'x_arr': list(range(1, 4))
        },
        # 9: 23.01.24, Config: 10/8/9 - tanh/tanh/tanh, B 10, E/LR (2500, 1500, 1000)/0.01, EF/NF (0, 1, 4)/0.05, generator is disabled
        '9': {
            'MSE_OD': [0.0054, 0.00635, 0.00542],
            'MSE_pH': [0.00125, 0.00124, 0.00101],
            'MSE_mrel': [0.01025, 0.01253, 0.01166],
            'R2_OD': [0.89733, 0.88543, 0.88643],
            'R2_pH': [0.74453, 0.7136, 0.7237],
            'R2_mrel': [0.7663, 0.73247, 0.75039],
            'x_arr': list(range(1, 3))
        }
    }
    # Post-processing
    stat_block = {}
    to_sum = (stat_blocks['e'],)
    for key in ('MSE_OD', 'MSE_pH', 'MSE_mrel', 'R2_OD', 'R2_pH', 'R2_mrel'):
        lines = []
        for item in to_sum:
            lines.append(item[key])
        stat_block[key] = [sum(x)/len(to_sum) for x in zip(*lines)]
    stat_block['x_arr'] = to_sum[0]['x_arr'] # Those should match among the things being summoned
    print(' done')
# End of __main__ section

In [ ]:
# Plotter for 'List' stat blocks
if __name__ == '__main__':
    import matplotlib.pyplot as plt
    import numpy as np
    x_array = stat_block['x_arr'] # range(hyperloop_count)
    x_poly = range(0, 20)
    #
    print('MSE graphs:')
    plt.figure(figsize=(16,10), dpi=100)
    polyval = True
    if polyval:
        plt.plot(x_poly, np.polyval(np.polyfit(x_array + [48, 49, 50], stat_block['MSE_OD'] + [0.006]*3, 6), x_poly), 'b-', label='OD\u2087\u2085\u2080')
        plt.plot(x_poly, np.polyval(np.polyfit(x_array + [48, 49, 50], stat_block['MSE_pH'] + [0.001]*3, 6), x_poly), 'r-', label='pH')
        plt.plot(x_poly, np.polyval(np.polyfit(x_array + [48, 49, 50], stat_block['MSE_mrel'] + [0.012]*3, 6), x_poly), 'g-', label='Уд. содерж. фикоцианина')
        #plt.plot(x_array, stat_block['MSE_OD'], 'b.', label='OD\u2087\u2085\u2080')
        #plt.plot(x_array, stat_block['MSE_pH'], 'r.', label='pH')
        #plt.plot(x_array, stat_block['MSE_mrel'], 'g.', label='Уд. содерж. фикоцианина')
        plt.xticks(ticks = (0, 5, 10, 15, 20))
    else:
        plt.plot(x_array, stat_block['MSE_OD'], 'b.-', label='OD\u2087\u2085\u2080')
        plt.plot(x_array, stat_block['MSE_pH'], 'r.-', label='pH')
        plt.plot(x_array, stat_block['MSE_mrel'], 'g.-', label='Уд. содерж. фикоцианина')
        plt.xticks(ticks = x_array)
    #plt.title('Зависимость среднеквадратичной ошибки от количества эпох обучения', fontsize=24)
    #plt.xlabel('Количество эпох обучения', fontsize=20)
    #plt.title('Зависимость среднеквадратичной ошибки от количества репликаций', fontsize=24)
    plt.xlabel('Количество репликаций', fontsize=20)
    plt.ylabel('Среднеквадратичная ошибка', fontsize=20)
    plt.legend(fontsize=16)
    plt.grid()
    plt.show()
    #
    print('R2 graphs:')
    plt.figure(figsize=(16,10), dpi=100)
    if polyval:
        plt.plot(x_poly, np.polyval(np.polyfit(x_array + [40, 50], stat_block['R2_OD'] + [0.8, 0.8], 6), x_poly), 'b-', label='OD\u2087\u2085\u2080')
        plt.plot(x_poly, np.polyval(np.polyfit(x_array + [40, 50], stat_block['R2_pH'] + [0.75, 0.75], 6), x_poly), 'r-', label='pH')
        plt.plot(x_poly, np.polyval(np.polyfit(x_array + [40, 50], stat_block['R2_mrel'] + [0.7, 0.7], 6), x_poly), 'g-', label='Уд. содерж. фикоцианина')
        #plt.plot(x_array, stat_block['R2_OD'], 'b.', label='OD\u2087\u2085\u2080')
        #plt.plot(x_array, stat_block['R2_pH'], 'r.', label='pH')
        #plt.plot(x_array, stat_block['R2_mrel'], 'g.', label='Уд. содерж. фикоцианина')
        plt.xticks(ticks = (0, 5, 10, 15, 20))
    else:
        plt.plot(x_array, stat_block['R2_OD'], 'b.-', label='OD\u2087\u2085\u2080')
        plt.plot(x_array, stat_block['R2_pH'], 'r.-', label='pH')
        plt.plot(x_array, stat_block['R2_mrel'], 'g.-', label='Уд. содерж. фикоцианина')
        plt.xticks(ticks = x_array)
    #plt.title('Зависимость коэффициента детерминации от количества эпох обучения', fontsize=24)
    #plt.xlabel('Количество эпох обучения', fontsize=20)
    #plt.title('Зависимость коэффициента детерминации от количества репликаций', fontsize=24)
    plt.xlabel('Количество репликаций', fontsize=20)
    plt.ylabel('Коэффициент детерминации', fontsize=20)
    plt.legend(fontsize=16)
    plt.grid()
    plt.show()
# End of __main__ section

In [ ]:
# Loader for 'Grid' stat blocks
if __name__ == '__main__':
    # Stat block example for testing:
    print("Loading \"Grid\" test stat block...", end='')
    stat_blocks = {
        # 1: 21.03.22, Config: 7/8/9 - sigmoid/sigmoid/sigmoid, B 50, E/LR 1000/0.01, EF/NF 14/0
        '1': {
            'MSE_OD': [0.00567, 0.0066, 0.00476, 0.00542, 0.00369, 0.00594, 0.00686, 0.00516, 0.00505, 0.00455, 0.00619, 0.00321, 0.00455, 0.00365, 0.00448, 0.00382, 0.00425, 0.00389, 0.00473, 0.00567, 0.00475, 0.00468, 0.00494, 0.00439, 0.00463, 0.00422, 0.00491, 0.00642, 0.00412, 0.00457, 0.00367, 0.00583, 0.00445, 0.00446, 0.00378, 0.00599, 0.00489, 0.00464, 0.00528, 0.00462, 0.00395, 0.00435, 0.00367, 0.00481, 0.00411, 0.00373, 0.00611, 0.00473, 0.00652, 0.00447, 0.00409, 0.00537, 0.00432, 0.00404, 0.0034, 0.00479, 0.0032, 0.00428, 0.00474, 0.0039, 0.00553, 0.0039, 0.00438, 0.00506, 0.00509, 0.0034, 0.00448, 0.00556, 0.00367, 0.00479, 0.00419, 0.00346, 0.00426, 0.00701, 0.00422, 0.00419, 0.00445, 0.01642, 0.01321, 0.01792, 0.00512, 0.00585, 0.00307, 0.00478, 0.00659, 0.00419, 0.00297, 0.00359, 0.00652, 0.00412, 0.00742, 0.00529, 0.00409, 0.00368, 0.0035, 0.00392, 0.00331, 0.0041, 0.00428, 0.00473, 0.00416],
            'MSE_pH': [0.00118, 0.00089, 0.00101, 0.00114, 0.00089, 0.00066, 0.00065, 0.00091, 0.00127, 0.00111, 0.00063, 0.00067, 0.00062, 0.00096, 0.00089, 0.00056, 0.00119, 0.00116, 0.00071, 0.00054, 0.00043, 0.00118, 0.00046, 0.00054, 0.0005, 0.00072, 0.00066, 0.00122, 0.00083, 0.00083, 0.00038, 0.00059, 0.00052, 0.0004, 0.00034, 0.00058, 0.00064, 0.00057, 0.00041, 0.00059, 0.00047, 0.00051, 0.0005, 0.00085, 0.00073, 0.00047, 0.00094, 0.0009, 0.00098, 0.00054, 0.00075, 0.00065, 0.0007, 0.00068, 0.00063, 0.00087, 0.00047, 0.00067, 0.00095, 0.00074, 0.00075, 0.00074, 0.00076, 0.00058, 0.00085, 0.00078, 0.0007, 0.00106, 0.00082, 0.00063, 0.00063, 0.00103, 0.00046, 0.00054, 0.00057, 0.00073, 0.00066, 0.00217, 0.0015, 0.00152, 0.00077, 0.00058, 0.0007, 0.00112, 0.00068, 0.00038, 0.00035, 0.00059, 0.00037, 0.00082, 0.00067, 0.00062, 0.00057, 0.00039, 0.00035, 0.00042, 0.0003, 0.00034, 0.00035, 0.00031, 0.00036],
            'MSE_mrel': [0.01401, 0.01453, 0.01547, 0.01046, 0.01402, 0.01244, 0.01211, 0.0109, 0.01077, 0.00991, 0.01304, 0.01182, 0.00883, 0.00976, 0.01358, 0.00978, 0.01177, 0.01253, 0.01197, 0.01256, 0.01421, 0.0103, 0.01303, 0.01631, 0.01772, 0.01189, 0.01426, 0.01119, 0.01153, 0.01787, 0.01441, 0.01124, 0.013, 0.00853, 0.01279, 0.01168, 0.01179, 0.01194, 0.01093, 0.01113, 0.00999, 0.01137, 0.01027, 0.01009, 0.01068, 0.00869, 0.01131, 0.01169, 0.01361, 0.00857, 0.01165, 0.01435, 0.01414, 0.01062, 0.00986, 0.01138, 0.01283, 0.00919, 0.0134, 0.01164, 0.01293, 0.01337, 0.01087, 0.00938, 0.01062, 0.00956, 0.01327, 0.01445, 0.01239, 0.01119, 0.01312, 0.01133, 0.01334, 0.0085, 0.01047, 0.01282, 0.0105, 0.03027, 0.0216, 0.02373, 0.01094, 0.00805, 0.00995, 0.00996, 0.02179, 0.01266, 0.01037, 0.00797, 0.00958, 0.008, 0.01446, 0.01564, 0.01128, 0.01163, 0.01147, 0.01061, 0.01092, 0.01022, 0.00947, 0.00867, 0.00845],
            'R2_OD': [0.85203, 0.82294, 0.86991, 0.85009, 0.9082, 0.8199, 0.86593, 0.87773, 0.87228, 0.88102, 0.88364, 0.90828, 0.89229, 0.90652, 0.89281, 0.88782, 0.86876, 0.89309, 0.85468, 0.86929, 0.86805, 0.88247, 0.86419, 0.87938, 0.88995, 0.89395, 0.87707, 0.83539, 0.90299, 0.88265, 0.89272, 0.84863, 0.8767, 0.86745, 0.90153, 0.85543, 0.87519, 0.8697, 0.88243, 0.85283, 0.90205, 0.86376, 0.89642, 0.88911, 0.90309, 0.91494, 0.85023, 0.90584, 0.83574, 0.87467, 0.90106, 0.87768, 0.88463, 0.89041, 0.89114, 0.885, 0.90481, 0.89325, 0.87948, 0.90564, 0.86877, 0.89941, 0.8873, 0.87994, 0.88018, 0.88523, 0.88849, 0.8702, 0.87881, 0.8712, 0.90952, 0.90075, 0.88511, 0.85989, 0.89983, 0.89465, 0.89079, 0.57639, 0.63178, 0.58834, 0.86967, 0.85289, 0.9004, 0.88896, 0.85327, 0.88319, 0.92658, 0.91845, 0.83045, 0.90659, 0.8597, 0.87957, 0.89001, 0.91719, 0.91467, 0.90449, 0.91529, 0.89723, 0.88792, 0.87509, 0.90221],
            'R2_pH': [0.52259, 0.55063, 0.54171, 0.5569, 0.64938, 0.55234, 0.68533, 0.67463, 0.47744, 0.4148, 0.71632, 0.73858, 0.77036, 0.52761, 0.60884, 0.73722, 0.43529, 0.48782, 0.68664, 0.76543, 0.78903, 0.59628, 0.79216, 0.74411, 0.78437, 0.66347, 0.6579, 0.51354, 0.64949, 0.64394, 0.76048, 0.6859, 0.76447, 0.7953, 0.80686, 0.74709, 0.71079, 0.74347, 0.81137, 0.73015, 0.77296, 0.75039, 0.75689, 0.67911, 0.71135, 0.75254, 0.70565, 0.56097, 0.56452, 0.7753, 0.69559, 0.61659, 0.64865, 0.69788, 0.71845, 0.61615, 0.75038, 0.62173, 0.56128, 0.64389, 0.52323, 0.61553, 0.50496, 0.77901, 0.60282, 0.51521, 0.70838, 0.56272, 0.56684, 0.70039, 0.69189, 0.55897, 0.77155, 0.73844, 0.78494, 0.72789, 0.70602, 0.14123, 0.3426, 0.34399, 0.61664, 0.7099, 0.64545, 0.57214, 0.55111, 0.83084, 0.84699, 0.58176, 0.80042, 0.64385, 0.6495, 0.66147, 0.76398, 0.82759, 0.86631, 0.81761, 0.85565, 0.78886, 0.80797, 0.81781, 0.83292],
            'R2_mrel': [0.76952, 0.74122, 0.71385, 0.81883, 0.69051, 0.80581, 0.7667, 0.79067, 0.78032, 0.83973, 0.74263, 0.77704, 0.84282, 0.83086, 0.7455, 0.77284, 0.76772, 0.77855, 0.73862, 0.81201, 0.73823, 0.83503, 0.78447, 0.71068, 0.53121, 0.79454, 0.73213, 0.75314, 0.79455, 0.71096, 0.74754, 0.79355, 0.76435, 0.83647, 0.76815, 0.79032, 0.75306, 0.7905, 0.83695, 0.78368, 0.79448, 0.77537, 0.79157, 0.80353, 0.81943, 0.8295, 0.79255, 0.81369, 0.75622, 0.84418, 0.76089, 0.73827, 0.78272, 0.79639, 0.826, 0.73622, 0.76556, 0.81209, 0.79975, 0.76675, 0.79822, 0.75269, 0.77718, 0.77558, 0.80139, 0.82847, 0.7472, 0.73035, 0.777, 0.7481, 0.76273, 0.79127, 0.7765, 0.83098, 0.79793, 0.75335, 0.81962, 0.41707, 0.56837, 0.50148, 0.76649, 0.85751, 0.80275, 0.80168, 0.64536, 0.66487, 0.82893, 0.83651, 0.79957, 0.8598, 0.72988, 0.72056, 0.81539, 0.80218, 0.79831, 0.78901, 0.79572, 0.80149, 0.79093, 0.83241, 0.85753]
        },
        # 2: 13.04.22, Config: 10/9/9 - sigmoid/sigmoid/sigmoid, B 25, E/LR 10000/0.01, EF/NF 9/0.01
        '2': {
            'MSE_OD': [0.00572, 0.00586, 0.00572, 0.00499, 0.00466, 0.00531, 0.0044, 0.00535, 0.00532, 0.005, 0.00369, 0.00723, 0.00453, 0.00394, 0.006, 0.00514, 0.00546, 0.00467, 0.00437, 0.00548, 0.00437, 0.00533, 0.00447, 0.00432, 0.00442, 0.00547, 0.00529, 0.00538, 0.00506, 0.00367, 0.00482, 0.00615, 0.00405, 0.00427, 0.00395, 0.00718, 0.00371, 0.00364, 0.00508, 0.00383, 0.00418, 0.00409, 0.00405, 0.00352, 0.0061, 0.00277, 0.00368, 0.00408, 0.00448, 0.00475, 0.00493, 0.00416, 0.00529, 0.00421, 0.00371, 0.00462, 0.00522, 0.00384, 0.00424, 0.00379, 0.00438, 0.0059, 0.0039, 0.00423, 0.00335, 0.00324, 0.00339, 0.00309, 0.00283, 0.00338, 0.00361, 0.00467, 0.00319, 0.00484, 0.00267, 0.00393, 0.00375, 0.00823, 0.00607, 0.00357, 0.00319, 0.00514, 0.00398, 0.00386, 0.00417, 0.00531, 0.00357, 0.00296, 0.0067, 0.00671, 0.00376, 0.00432, 0.00749, 0.00347, 0.00363, 0.00309, 0.00651, 0.00277, 0.00539, 0.00287, 0.00519],
            'MSE_pH': [0.00099, 0.00116, 0.00109, 0.00078, 0.00092, 0.00077, 0.00088, 0.00065, 0.00079, 0.0012, 0.00096, 0.00108, 0.00126, 0.00115, 0.00111, 0.00089, 0.00126, 0.001, 0.00117, 0.00092, 0.00061, 0.00065, 0.00065, 0.00044, 0.00061, 0.00094, 0.00086, 0.00138, 0.00071, 0.00059, 0.00038, 0.00083, 0.00067, 0.00041, 0.00047, 0.00054, 0.00051, 0.00042, 0.00039, 0.00052, 0.00038, 0.00039, 0.00055, 0.00066, 0.00057, 0.00043, 0.00041, 0.00045, 0.00044, 0.00048, 0.0005, 0.00056, 0.0005, 0.00048, 0.00042, 0.00056, 0.00045, 0.00038, 0.00063, 0.0006, 0.00037, 0.00041, 0.00041, 0.00044, 0.00037, 0.00033, 0.00057, 0.00036, 0.0004, 0.00043, 0.00043, 0.00041, 0.00033, 0.00036, 0.00034, 0.00043, 0.00032, 0.00159, 0.00067, 0.0004, 0.00032, 0.00046, 0.00038, 0.00057, 0.00054, 0.0003, 0.00031, 0.00042, 0.00083, 0.00169, 0.00038, 0.00038, 0.00035, 0.00044, 0.00044, 0.00045, 0.00051, 0.00034, 0.00033, 0.00049, 0.00036],
            'MSE_mrel': [0.01366, 0.01773, 0.01906, 0.01246, 0.01325, 0.01073, 0.012, 0.01327, 0.01502, 0.01462, 0.00987, 0.01089, 0.01136, 0.00906, 0.00982, 0.0092, 0.01237, 0.01267, 0.01219, 0.01056, 0.01353, 0.0103, 0.01117, 0.01112, 0.01699, 0.01273, 0.01139, 0.01277, 0.01101, 0.01026, 0.0109, 0.01566, 0.01028, 0.00977, 0.01133, 0.01266, 0.01013, 0.01223, 0.01178, 0.01002, 0.00875, 0.01033, 0.01146, 0.0079, 0.01081, 0.01156, 0.01096, 0.00794, 0.00983, 0.01114, 0.01258, 0.01107, 0.01003, 0.01088, 0.01216, 0.01245, 0.00975, 0.0087, 0.01167, 0.00958, 0.01145, 0.00961, 0.01069, 0.01014, 0.01089, 0.01176, 0.01334, 0.01228, 0.01181, 0.00841, 0.01068, 0.01155, 0.00954, 0.01021, 0.01046, 0.00987, 0.0107, 0.01246, 0.01196, 0.0082, 0.00996, 0.01269, 0.01227, 0.01213, 0.00823, 0.01437, 0.00946, 0.0076, 0.0173, 0.0144, 0.01432, 0.01084, 0.01073, 0.00828, 0.01099, 0.01902, 0.01464, 0.00999, 0.01009, 0.00841, 0.01101],
            'R2_OD': [0.86808, 0.83713, 0.86995, 0.867, 0.89215, 0.89712, 0.8706, 0.8759, 0.86116, 0.8238, 0.89163, 0.80797, 0.88915, 0.89132, 0.82428, 0.8902, 0.8733, 0.87629, 0.88893, 0.86544, 0.89459, 0.87785, 0.87969, 0.89257, 0.89631, 0.8533, 0.86192, 0.85158, 0.88748, 0.91308, 0.87416, 0.81181, 0.88881, 0.89733, 0.88032, 0.81121, 0.89862, 0.92265, 0.87319, 0.90598, 0.91387, 0.90646, 0.88683, 0.91377, 0.86466, 0.93756, 0.91782, 0.90053, 0.87118, 0.87165, 0.85483, 0.89997, 0.87129, 0.8946, 0.91726, 0.8446, 0.87631, 0.86466, 0.88702, 0.88554, 0.89665, 0.85001, 0.88539, 0.89407, 0.92422, 0.92419, 0.92014, 0.92568, 0.92258, 0.91158, 0.91684, 0.87926, 0.92646, 0.8504, 0.92933, 0.91724, 0.89986, 0.83147, 0.82747, 0.89982, 0.9348, 0.87106, 0.88372, 0.92062, 0.88297, 0.85558, 0.92298, 0.90497, 0.84523, 0.84226, 0.89887, 0.91225, 0.83167, 0.92701, 0.92864, 0.92543, 0.81977, 0.93447, 0.87184, 0.93446, 0.89697],
            'R2_pH': [0.48701, 0.47669, 0.52587, 0.61578, 0.61412, 0.53817, 0.55963, 0.68494, 0.5882, 0.5043, 0.60533, 0.64044, 0.49762, 0.59711, 0.56071, 0.55859, 0.49501, 0.58132, 0.41599, 0.59455, 0.68212, 0.75558, 0.64295, 0.78454, 0.7612, 0.68157, 0.66685, 0.47328, 0.72826, 0.68456, 0.78954, 0.68214, 0.68713, 0.82628, 0.74328, 0.70832, 0.67718, 0.83228, 0.8064, 0.73368, 0.80966, 0.83336, 0.73077, 0.73867, 0.77411, 0.79991, 0.81285, 0.83108, 0.77323, 0.76839, 0.72161, 0.73744, 0.75792, 0.80069, 0.80681, 0.72734, 0.81765, 0.82317, 0.71764, 0.68031, 0.82977, 0.82617, 0.84327, 0.79443, 0.83461, 0.80023, 0.64513, 0.8219, 0.79761, 0.83599, 0.82377, 0.82709, 0.86983, 0.83826, 0.8143, 0.81755, 0.7938, 0.30929, 0.70095, 0.82898, 0.85281, 0.75625, 0.80842, 0.78094, 0.79672, 0.82687, 0.88669, 0.80475, 0.58373, 0.34298, 0.83715, 0.82875, 0.86021, 0.84748, 0.82331, 0.78618, 0.68886, 0.81958, 0.82872, 0.77138, 0.76011],
            'R2_mrel': [0.78732, 0.72623, 0.71037, 0.73742, 0.76851, 0.80319, 0.7816, 0.76354, 0.76825, 0.74123, 0.82611, 0.8035, 0.75442, 0.80833, 0.84346, 0.82564, 0.75697, 0.77292, 0.75287, 0.79967, 0.72046, 0.79465, 0.77196, 0.78876, 0.68523, 0.74253, 0.76296, 0.77144, 0.79909, 0.80195, 0.7948, 0.70362, 0.81459, 0.81138, 0.77318, 0.74135, 0.79655, 0.81674, 0.7985, 0.81211, 0.84758, 0.74848, 0.79296, 0.86387, 0.79146, 0.8084, 0.77916, 0.8595, 0.83128, 0.76943, 0.77691, 0.78915, 0.79723, 0.78696, 0.77846, 0.77012, 0.81643, 0.83097, 0.81005, 0.83211, 0.80511, 0.82148, 0.8017, 0.78406, 0.78345, 0.78618, 0.74971, 0.76222, 0.80475, 0.83171, 0.80934, 0.78931, 0.8202, 0.8299, 0.77372, 0.82844, 0.78089, 0.74903, 0.80131, 0.84204, 0.78195, 0.7636, 0.78711, 0.80682, 0.86026, 0.73835, 0.81104, 0.85881, 0.71773, 0.698, 0.73099, 0.82166, 0.77212, 0.85239, 0.80755, 0.6417, 0.68498, 0.82425, 0.82952, 0.85239, 0.79518]
        },
        # 3: 11.12.22, failed run
        '3': {
            'MSE_OD': [0.00672, 0.00769, 0.00903, 0.00593, 0.00438, 0.00587, 0.00571, 0.00532, 0.00742, 0.00688, 0.00392, 0.00534, 0.00772, 0.0052, 0.00544, 0.00496, 0.00497, 0.00642, 0.00645, 0.00548, 0.00458, 0.00537, 0.00509, 0.00469, 0.00517, 0.00461, 0.00458, 0.00495, 0.00635, 0.00643, 0.00495, 0.00392, 0.0043, 0.00518, 0.0043, 0.00555, 0.007, 0.01349, 0.00409, 0.0051, 0.0038, 0.00361, 0.00419, 0.00325, 0.00398, 0.00265, 0.00336, 0.00493, 0.00423],
            'MSE_pH': [0.0018, 0.00179, 0.00146, 0.00151, 0.00137, 0.00116, 0.00132, 0.00128, 0.00153, 0.00189, 0.00126, 0.00127, 0.00164, 0.00157, 0.00145, 0.00122, 0.00187, 0.0017, 0.00122, 0.00134, 0.0011, 0.00127, 0.00095, 0.00116, 0.00135, 0.00117, 0.00087, 0.00092, 0.00122, 0.00103, 0.00116, 0.00104, 0.00088, 0.00073, 0.00082, 0.00094, 0.00147, 0.00231, 0.00088, 0.00063, 0.0008, 0.00072, 0.00072, 0.00071, 0.00065, 0.00081, 0.00098, 0.00097, 0.0008],
            'MSE_mrel': [0.01439, 0.01427, 0.01605, 0.01419, 0.01259, 0.01642, 0.01371, 0.01284, 0.01445, 0.01054, 0.01215, 0.01195, 0.01134, 0.01391, 0.01274, 0.01225, 0.01191, 0.01124, 0.01672, 0.01161, 0.01172, 0.01319, 0.01527, 0.01071, 0.01248, 0.00884, 0.01027, 0.01206, 0.01323, 0.01541, 0.01107, 0.01104, 0.00909, 0.01058, 0.01332, 0.01491, 0.01211, 0.02561, 0.01235, 0.0101, 0.00984, 0.00963, 0.0102, 0.0084, 0.01158, 0.00793, 0.00851, 0.01207, 0.01014],
            'R2_OD': [0.88028, 0.84027, 0.81037, 0.88527, 0.89945, 0.88536, 0.89593, 0.87519, 0.84698, 0.88372, 0.89845, 0.89394, 0.85146, 0.88816, 0.89786, 0.90082, 0.89098, 0.86789, 0.87009, 0.8961, 0.90716, 0.90985, 0.89653, 0.91301, 0.88108, 0.91215, 0.89916, 0.8949, 0.87618, 0.8803, 0.89317, 0.926, 0.89937, 0.89798, 0.91968, 0.90453, 0.87176, 0.74838, 0.92048, 0.90126, 0.92321, 0.92199, 0.91991, 0.93557, 0.92749, 0.94255, 0.93168, 0.87797, 0.92141],
            'R2_pH': [0.56011, 0.58409, 0.64125, 0.70541, 0.6746, 0.7233, 0.68718, 0.66227, 0.64885, 0.61817, 0.72966, 0.69447, 0.61035, 0.69022, 0.6994, 0.71164, 0.54256, 0.62431, 0.72267, 0.68676, 0.75105, 0.71468, 0.79443, 0.71959, 0.66434, 0.74211, 0.80651, 0.7839, 0.70851, 0.75677, 0.77551, 0.75726, 0.79328, 0.81199, 0.81614, 0.78073, 0.65711, 0.49119, 0.81272, 0.84547, 0.80451, 0.81986, 0.82061, 0.8236, 0.84315, 0.81461, 0.7831, 0.78051, 0.81389],
            'R2_mrel': [0.67907, 0.66965, 0.6357, 0.67985, 0.73489, 0.64906, 0.68194, 0.72248, 0.70879, 0.7533, 0.68924, 0.69979, 0.74494, 0.70596, 0.71696, 0.72931, 0.7203, 0.74808, 0.6585, 0.7182, 0.75242, 0.64409, 0.69025, 0.71392, 0.71921, 0.80159, 0.742, 0.73498, 0.71578, 0.67475, 0.77379, 0.78022, 0.80126, 0.77547, 0.73934, 0.66459, 0.70667, 0.47592, 0.71448, 0.79364, 0.79072, 0.76967, 0.77593, 0.81962, 0.76046, 0.84323, 0.8135, 0.7573, 0.75857]
        },
        # 4: 12.12.22, Config: 5/7/9 - sigmoid/sigmoid/sigmoid, B 50, E/LR 2500/0.025, EF/NF 4/0.03
        '4': {
            'MSE_OD': [0.0074, 0.0068, 0.00618, 0.0065, 0.00911, 0.00627, 0.00555, 0.00525, 0.00557, 0.0052, 0.00533, 0.00625, 0.00479, 0.00504, 0.00463, 0.00567, 0.00713, 0.00594, 0.00599, 0.00549, 0.0038, 0.00511, 0.00423, 0.00868, 0.006, 0.00516, 0.00484, 0.00405, 0.00381, 0.0059, 0.00563, 0.0044, 0.00657, 0.00577, 0.00441, 0.00467, 0.00388, 0.00528, 0.00571, 0.00431, 0.00549, 0.00398, 0.00489, 0.00401, 0.00446, 0.00643, 0.00365, 0.00396, 0.00418, 0.00604, 0.00532, 0.00417, 0.00434, 0.00331, 0.00575, 0.00394, 0.00482, 0.00553, 0.00503, 0.00349, 0.00434, 0.0042, 0.004, 0.00406, 0.00401, 0.00511, 0.0066, 0.00321, 0.00452, 0.00312, 0.00377, 0.005, 0.00437],
            'MSE_pH': [0.00143, 0.00141, 0.00139, 0.00144, 0.00172, 0.00141, 0.0013, 0.00106, 0.00163, 0.00106, 0.00108, 0.00137, 0.00141, 0.00098, 0.00102, 0.00106, 0.0011, 0.00089, 0.00143, 0.00113, 0.00095, 0.0009, 0.00089, 0.00178, 0.001, 0.00089, 0.00092, 0.00138, 0.001, 0.0013, 0.0012, 0.00105, 0.00122, 0.00128, 0.00102, 0.00088, 0.00092, 0.00107, 0.00131, 0.00104, 0.00111, 0.00165, 0.00104, 0.00103, 0.00064, 0.00155, 0.00073, 0.00078, 0.00067, 0.00109, 0.00079, 0.00081, 0.00067, 0.00053, 0.00099, 0.00064, 0.00116, 0.00079, 0.00081, 0.00054, 0.00095, 0.00072, 0.00057, 0.00094, 0.00085, 0.00081, 0.0006, 0.00078, 0.00077, 0.00087, 0.0009, 0.00084, 0.00082],
            'MSE_mrel': [0.01455, 0.01525, 0.01114, 0.0122, 0.01302, 0.01134, 0.01395, 0.01169, 0.01097, 0.01347, 0.01372, 0.01236, 0.0123, 0.01028, 0.01196, 0.01245, 0.01371, 0.01304, 0.01297, 0.01042, 0.01064, 0.01349, 0.01169, 0.01646, 0.01235, 0.00957, 0.01041, 0.0079, 0.00837, 0.01159, 0.01502, 0.01162, 0.01247, 0.0125, 0.00985, 0.00947, 0.00935, 0.00892, 0.01104, 0.01119, 0.01108, 0.00921, 0.00848, 0.01102, 0.01145, 0.01273, 0.0097, 0.00863, 0.00955, 0.01499, 0.01408, 0.01079, 0.01012, 0.00947, 0.01132, 0.00811, 0.01124, 0.01134, 0.0094, 0.00876, 0.01134, 0.00977, 0.00943, 0.01013, 0.01097, 0.01248, 0.01111, 0.00997, 0.01057, 0.01052, 0.01051, 0.00997, 0.00887],
            'R2_OD': [0.86444, 0.86733, 0.8888, 0.88498, 0.81681, 0.87285, 0.87657, 0.90359, 0.89224, 0.89434, 0.88078, 0.86999, 0.89752, 0.90243, 0.90887, 0.9058, 0.86652, 0.87872, 0.88139, 0.90609, 0.92383, 0.90469, 0.91123, 0.83205, 0.87866, 0.89778, 0.91511, 0.92512, 0.93068, 0.8874, 0.89193, 0.9088, 0.88064, 0.91016, 0.90437, 0.91179, 0.91072, 0.88485, 0.88578, 0.91345, 0.9001, 0.91613, 0.8947, 0.92771, 0.91462, 0.86278, 0.92977, 0.92422, 0.91397, 0.89225, 0.89867, 0.91797, 0.90718, 0.9303, 0.90569, 0.91221, 0.90268, 0.88195, 0.90231, 0.92617, 0.91499, 0.92457, 0.9088, 0.91587, 0.91794, 0.89427, 0.8795, 0.94131, 0.91958, 0.92741, 0.91923, 0.89332, 0.92493],
            'R2_pH': [0.67088, 0.662, 0.64066, 0.652, 0.60606, 0.70986, 0.71634, 0.73919, 0.59179, 0.73181, 0.71931, 0.68091, 0.66636, 0.75281, 0.72469, 0.77158, 0.69474, 0.7968, 0.70623, 0.69326, 0.76172, 0.79285, 0.78346, 0.55305, 0.77465, 0.81229, 0.77514, 0.70208, 0.79919, 0.68527, 0.71701, 0.75036, 0.76179, 0.7246, 0.77846, 0.77556, 0.76123, 0.71554, 0.67867, 0.74195, 0.72063, 0.62505, 0.74088, 0.74641, 0.84975, 0.63811, 0.82859, 0.83949, 0.83805, 0.75745, 0.79964, 0.81184, 0.83607, 0.84122, 0.79495, 0.82538, 0.70811, 0.79188, 0.7954, 0.86027, 0.77129, 0.82381, 0.85374, 0.78159, 0.81787, 0.79812, 0.83217, 0.82931, 0.8058, 0.79543, 0.79798, 0.77523, 0.80068],
            'R2_mrel': [0.67284, 0.65548, 0.73896, 0.69592, 0.7209, 0.73639, 0.72548, 0.74038, 0.74855, 0.68492, 0.66409, 0.73266, 0.71975, 0.76863, 0.73878, 0.67311, 0.68998, 0.70147, 0.72437, 0.75706, 0.76277, 0.6897, 0.72179, 0.58279, 0.70807, 0.77378, 0.78588, 0.79134, 0.81413, 0.72904, 0.66651, 0.71516, 0.71214, 0.72743, 0.77616, 0.77758, 0.79694, 0.76866, 0.7555, 0.76428, 0.72422, 0.79967, 0.79725, 0.74964, 0.75472, 0.65576, 0.78383, 0.80715, 0.76941, 0.66144, 0.66315, 0.77036, 0.7852, 0.75669, 0.76164, 0.80262, 0.73122, 0.74235, 0.73856, 0.76969, 0.7542, 0.79595, 0.79364, 0.71882, 0.75703, 0.73649, 0.7539, 0.77264, 0.7595, 0.7626, 0.7663, 0.78821, 0.78462],
        },
        # 5: 26.01.24,         [5, 7, 10, 15, 25]x3, ['sigmoid', 'tanh', 'relu']x3, [10, 40, 100], [500, 1500, 5000], [0.001, 0.005, 0.025, 0.1], [0, 1, 4], [0, 0.01, 0.05]
        '5': {
            'MSE_OD': [0.00644, 0.00689, 0.00458, 0.00409, 0.00443, 0.00469, 0.00447, 0.00517, 0.00498, 0.00507, 0.00486, 0.00434, 0.00548, 0.00317, 0.00423, 0.00496, 0.00423, 0.00658, 0.00356, 0.00495, 0.00479, 0.00363, 0.00464, 0.00536, 0.00789, 0.00422, 0.00339, 0.00555, 0.00342, 0.00364, 0.00533, 0.00508, 0.00255, 0.01012, 0.00351, 0.00259, 0.00428, 0.00277, 0.00258, 0.00224, 0.00292, 0.0026, 0.00264, 0.00315, 0.00229, 0.00398, 0.00404, 0.00377, 0.00292, 0.00292, 0.00351, 0.00292, 0.00344, 0.00301, 0.00277, 0.00313, 0.00556, 0.05327, 0.0027, 0.00301, 0.00505, 0.00397, 0.00354, 0.00378, 0.01315, 0.00387, 0.00354, 0.00582, 0.00383, 0.00248, 0.00739, 0.00444, 0.00278, 0.02633, 0.0033, 0.00355, 0.00296, 0.00359, 0.00315, 0.00363, 0.00294],
            'MSE_pH': [0.00091, 0.00104, 0.00073, 0.00075, 0.00053, 0.00079, 0.00082, 0.0006, 0.00074, 0.00058, 0.00081, 0.00054, 0.00063, 0.00052, 0.00064, 0.00066, 0.00054, 0.00057, 0.00053, 0.00099, 0.00119, 0.00047, 0.00091, 0.00219, 0.00149, 0.00064, 0.00042, 0.0009, 0.00057, 0.00039, 0.00063, 0.00047, 0.00039, 0.00247, 0.00049, 0.00037, 0.00037, 0.00048, 0.00053, 0.00048, 0.00045, 0.00048, 0.00045, 0.00077, 0.00046, 0.00075, 0.00053, 0.00058, 0.00051, 0.00048, 0.00068, 0.00054, 0.00071, 0.00057, 0.00058, 0.00062, 0.00077, 0.00501, 0.00034, 0.00051, 0.00109, 0.00043, 0.0006, 0.00081, 0.00201, 0.00056, 0.00056, 0.00071, 0.00046, 0.00045, 0.00123, 0.00053, 0.00044, 0.00346, 0.00052, 0.00052, 0.00046, 0.00053, 0.00038, 0.00059, 0.00045],
            'MSE_mrel': [0.01063, 0.01133, 0.012, 0.01145, 0.00881, 0.01107, 0.01058, 0.01037, 0.01053, 0.00761, 0.00918, 0.01036, 0.00935, 0.00912, 0.00889, 0.0102, 0.0081, 0.01035, 0.01307, 0.01052, 0.01487, 0.00846, 0.01114, 0.03487, 0.01179, 0.00892, 0.0101, 0.01122, 0.00866, 0.00676, 0.01166, 0.00717, 0.00963, 0.02099, 0.00882, 0.0087, 0.00684, 0.00887, 0.00634, 0.00727, 0.00733, 0.00758, 0.00618, 0.00706, 0.00855, 0.00912, 0.00872, 0.00869, 0.0068, 0.00742, 0.00771, 0.00798, 0.0074, 0.00838, 0.00685, 0.00836, 0.00739, 0.04242, 0.0085, 0.00697, 0.01088, 0.00832, 0.00942, 0.01055, 0.01864, 0.0088, 0.00818, 0.01274, 0.00922, 0.0056, 0.01377, 0.01063, 0.00831, 0.02798, 0.01181, 0.00783, 0.0093, 0.00803, 0.00783, 0.00783, 0.00788],
            'R2_OD': [0.8646, 0.86252, 0.91093, 0.91167, 0.89897, 0.90759, 0.91472, 0.88686, 0.90499, 0.89611, 0.9022, 0.92085, 0.89402, 0.93798, 0.9121, 0.9071, 0.91453, 0.86749, 0.93021, 0.90424, 0.90576, 0.92254, 0.9238, 0.90038, 0.86149, 0.92198, 0.9322, 0.8852, 0.93124, 0.92833, 0.87899, 0.89293, 0.94298, 0.79136, 0.9354, 0.94677, 0.93333, 0.94312, 0.95254, 0.95497, 0.94938, 0.95264, 0.95064, 0.93888, 0.95848, 0.91438, 0.91765, 0.92227, 0.93768, 0.93907, 0.9329, 0.94159, 0.93694, 0.94494, 0.94413, 0.92698, 0.8969, -0.00902, 0.94621, 0.92962, 0.89719, 0.90628, 0.92738, 0.92817, 0.75386, 0.9243, 0.93267, 0.88734, 0.92027, 0.95323, 0.85804, 0.90091, 0.92759, 0.52651, 0.93166, 0.92923, 0.94884, 0.9398, 0.93912, 0.93272, 0.9407],
            'R2_pH': [0.81123, 0.76371, 0.81115, 0.85288, 0.86572, 0.82132, 0.81782, 0.84992, 0.846, 0.87567, 0.82955, 0.86276, 0.85953, 0.88411, 0.8757, 0.83899, 0.88879, 0.85681, 0.87328, 0.8003, 0.76366, 0.8959, 0.80807, 0.51952, 0.70109, 0.86873, 0.90256, 0.7518, 0.87713, 0.92521, 0.85806, 0.90974, 0.91165, 0.46713, 0.881, 0.91896, 0.90517, 0.88354, 0.88443, 0.89422, 0.89086, 0.89428, 0.90622, 0.81502, 0.90619, 0.80156, 0.8859, 0.87799, 0.89288, 0.90613, 0.86478, 0.87235, 0.83231, 0.88131, 0.88007, 0.86789, 0.83735, -0.00486, 0.91912, 0.88538, 0.74073, 0.9008, 0.88255, 0.83284, 0.57698, 0.85943, 0.86634, 0.83463, 0.89757, 0.91179, 0.76569, 0.8825, 0.90467, 0.1947, 0.87675, 0.88929, 0.89362, 0.88421, 0.90404, 0.87348, 0.90748],
            'R2_mrel': [0.72049, 0.69372, 0.72569, 0.75829, 0.77395, 0.75908, 0.75906, 0.74232, 0.72142, 0.82285, 0.79706, 0.72646, 0.78164, 0.78551, 0.76814, 0.73583, 0.82195, 0.77243, 0.68083, 0.75223, 0.61336, 0.81185, 0.77402, 0.03545, 0.70922, 0.77862, 0.7558, 0.73048, 0.74247, 0.8188, 0.73405, 0.83065, 0.80511, 0.54478, 0.78602, 0.78934, 0.84828, 0.74838, 0.85158, 0.82646, 0.84451, 0.81397, 0.80865, 0.83081, 0.79381, 0.7992, 0.79367, 0.76737, 0.84119, 0.788, 0.78921, 0.75639, 0.81729, 0.79206, 0.84227, 0.816, 0.80025, -0.00696, 0.81505, 0.80172, 0.73335, 0.80264, 0.76203, 0.73249, 0.51922, 0.76019, 0.83346, 0.70045, 0.7531, 0.86065, 0.68575, 0.765, 0.80546, 0.23978, 0.73004, 0.81786, 0.77855, 0.8207, 0.80679, 0.79968, 0.78259]
        }
    }
    # Post-processing
    stat_block = {}
    to_sum = (stat_blocks['4'],)
    for key in ('MSE_OD', 'MSE_pH', 'MSE_mrel', 'R2_OD', 'R2_pH', 'R2_mrel'):
        lines = []
        for item in to_sum:
            lines.append(item[key])
        stat_block[key] = [sum(x)/len(to_sum) for x in zip(*lines)]
    hyperloop_count = len(stat_block['MSE_OD'])
    print(' done')
# End of __name__ section

In [ ]:
# Plotter for 'Grid' stat blocks
if __name__ == '__main__':
    import matplotlib.pyplot as plt
    import numpy as np
    x_array = list(x+1 for x in list(range(hyperloop_count)))
    print('MSE graphs:')
    plt.figure(figsize=(16,10), dpi=100)
    plt.plot(x_array, stat_block['MSE_OD'], 'b.-', label='OD\u2087\u2085\u2080')
    plt.plot(x_array, stat_block['MSE_pH'], 'r.-', label='pH')
    plt.plot(x_array, stat_block['MSE_mrel'], 'g.-', label='Уд. содерж. фикоцианина')
    #plt.xticks(ticks = x_array)
    #plt.title('Cреднеквадратичная ошибка для различных конфигураций гиперпараметров', fontsize=20)
    plt.xlabel('Конфигурация гиперпараметров ИНС', fontsize=20)
    plt.ylabel('Среднеквадратичная ошибка', fontsize=20)
    plt.legend(fontsize=16)
    plt.grid()
    plt.show()
    print('R2 graphs:')
    plt.figure(figsize=(16,10), dpi=100)
    plt.plot(x_array, stat_block['R2_OD'], 'b.-', label='OD\u2087\u2085\u2080')
    plt.plot(x_array, stat_block['R2_pH'], 'r.-', label='pH')
    plt.plot(x_array, stat_block['R2_mrel'], 'g.-', label='Уд. содерж. фикоцианина')
    #plt.xticks(ticks = x_array)
    #plt.title('Коэффициент детерминации для различных конфигураций гиперпараметров', fontsize=20)
    plt.xlabel('Конфигурация гиперпараметров ИНС', fontsize=20)
    plt.ylabel('Коэффициент детерминации', fontsize=20)
    plt.legend(fontsize=16)
    plt.grid()
    plt.show()
# End of __name__ section

<div class="alert alert-block alert-info"><b>The model trainer and model caller - for running predictions. Should be stand-alone.</b></div>

In [ ]:
import os

import pandas as pd
import numpy as np
import tensorflow as tf
#from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.models import load_model

from itertools import cycle
from tqdm.keras import TqdmCallback

#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"   # Disable GPU if necessary

In [ ]:
def noisy_duplicate_keras_b(data_array_loc): # alt version, might fix the memory issue
    data_array_new = data_array_loc.copy()
    # Element-by-element
    for i in range(trainset_length):
        for j in range(input_num_units):
            data_array_new[i][j] = data_array_new[i][j]*(1 + noise_factor*(0.5 - np.random.random_sample()))
    return data_array_new

def hp_config_print(hp_list):
    label_list = ["Config: ", "/", "/", " - ", "/", "/", ", B ", ", E/LR ", "/", ", EF/NF ", "/"]
    for item, label in zip(hp_list, label_list): print(label, item, sep='', end='')
    print()

def trainset_generator(x_array, y_array, batch_size):
    pairs = [(x, y) for x in x_array for y in y_array]
    cycle_pairs = cycle(pairs)
    while (True):
        x_batch = []
        y_batch = []
        for _ in range(batch_size):
            x, y = next(cycle_pairs)
            x_batch.append(x)
            y_batch.append(y)
        yield np.array(x_batch), np.array(y_batch)

In [ ]:
# Model trainer
if __name__ == '__main__':
    print("Initialising...")
    work_path = os.path.join("k:" + os.sep, "Archive", "Studying", "NeuralNets", "Python", "Jupyter - Phycocyanin ANN")
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus: tf.config.experimental.set_memory_growth(gpu, True)
    marker = 'C'
    generator_enabled = True
    low_priority_enabled = False
    if low_priority_enabled:
        import psutil
        this_process = psutil.Process(os.getpid())
        this_process.nice(psutil.BELOW_NORMAL_PRIORITY_CLASS)
    # Set RNG
    seed = 128
    rng = np.random.RandomState(seed)
    tf.random.set_seed(seed) # Perhaps I need to set tf. seed separately?
    # Static parameters
    dataset_length = 490 # Out of 504 total
    train_list_length = 'F'
    input_num_units = 12 #
    output_num_units = 3 # 3 is all
    # Reading data
    df = pd.read_csv(os.path.join(work_path, "Exp data SE 7 (good-504).csv"))
    df.info()
    df.drop(columns=['No', 'Init OD680', 'OD680'], inplace=True) # Drop the ID and 680s
    df = df.replace(to_replace='???', value=np.NaN) # Replace '???'s with true NaNs for ease of processing
    # Replace NaNs with means
    for i in df.columns[df.isnull().any(axis=0)]:
        df[i] = df[i].astype(np.float64) # Convert to float64 explicitly since there were string objects there
        df[i].fillna(df[i].mean(),inplace=True)
    mean_val = (df[df.columns.to_list()].mean()).to_numpy() # Not needed, testing only
    max_val = (df[df.columns.to_list()].max()) # Divide wont' work with a np_array of max values
    for i in df.columns:
        df[i] = df[i].div(max_val[i]) # Divide one by one because if not, the columns will get sorted alphabetically
    max_val = max_val.to_numpy()
    # Now, make it into a numpy array
    dataset = df.to_numpy()
    del df
    #print("Max values are:\n", np.around(max_val, decimals=5))
    #print("Mean values are:\n", np.around(mean_val, decimals=5))
    #print("Processed dataset example:\n", dataset[55])
    print("Dataset shape is:", dataset.shape)
    # Hyperparameters
    # Current best: [10, 8, 9, 'sigmoid', 'sigmoid', 'sigmoid', 20, 4000, 0.001, 4, 0.05]
    hyperparameters = [10, 8, 9, 'sigmoid', 'sigmoid', 'sigmoid', 20, 4000, 0.001, 4, 0.05]
    print("Starting the process...")
    #
    hidden1_num_units, hidden2_num_units, hidden3_num_units, hidden1_func, hidden2_func, hidden3_func, batch_size, epochs, learning_rate, extension_factor, noise_factor = hyperparameters
    hp_config_print(hyperparameters)
    # Data normalization for the noise
    dataset_norm = (dataset.copy())/(1 + noise_factor)  # Legacy
    # Keras: creating a model for the set of hyperparameters to be used for a single stat loop
    # The model is then cleared by clear_session
    model = Sequential([
        Dense(units=hidden1_num_units, input_shape=(input_num_units,), activation=hidden1_func),
        Dense(units=hidden2_num_units, activation=hidden2_func),
        Dense(units=hidden3_num_units, activation=hidden3_func),
        Dense(units=output_num_units, activation='linear')
    ])
    #model.summary() # Debug
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=['mse'])
    # Randomising data selection: grab the whole set and shuffle it
    # I can't use Keras shuffle and validation split because of the inflation algorithm
    # This way test set is different every time and randomly selected while still being X pre-selected numbers
    trainset_length = dataset_length
    train_in_array = np.zeros((trainset_length, input_num_units))
    train_out_array = np.zeros((trainset_length, output_num_units))
    #
    #rand_list = np.random.choice(trainset_length, trainset_length, False)
    for i in range(trainset_length):
        line_to_add = dataset_norm[i]
        train_in_array[i] = (line_to_add[:input_num_units].copy())/(1 + noise_factor)
        train_out_array[i] = (line_to_add[input_num_units:].copy())
    # Extending dataset with 'noisy' duplicates
    train_in_array_temp = train_in_array.copy()
    train_out_array_temp = train_out_array.copy()
    for i in range(0, extension_factor):
        train_in_array_temp = np.concatenate((train_in_array_temp, noisy_duplicate_keras_b(train_in_array)))  # Noisy inputs
        train_out_array_temp = np.concatenate((train_out_array_temp, train_out_array))                      # Clean corresponding outputs
    train_in_array = train_in_array_temp.copy()    # Copy because I'm clearing it, which is probably unnecessary
    train_out_array = train_out_array_temp.copy()
    del train_in_array_temp
    del train_out_array_temp
    #
    train_list_length = dataset_length*(1 + extension_factor)   # Legacy
    if len(train_in_array) != train_list_length: print("--- Train list length is off! ---")
    # train_list = dataset[:dataset_length]
    # test_list = dataset[dataset_length:]
    #print("Train input shape is:", train_in_array.shape, "train output shape is:", train_out_array.shape)
    #print("Test input shape is:", test_in_array.shape, "test output shape is:", test_out_array.shape)
    # Calling the Keras model to train
    # Fit: suffle=False because it is already shuffled because of how the dataset is split between train and test
    #print("Doing fit...")
    if generator_enabled:
        trainset = trainset_generator(train_in_array, train_out_array, batch_size)
        model.fit(trainset, steps_per_epoch = len(train_in_array)/batch_size, batch_size=batch_size, epochs=epochs, shuffle=False, verbose=0, callbacks=[TqdmCallback(verbose=0)])
    else:
        model.fit(x=train_in_array, y=train_out_array, batch_size=batch_size, epochs=epochs, shuffle=False, verbose=0, callbacks=[TqdmCallback(verbose=0)]) # Callback does slow things down somewhat
    #print("Fit done")
    print("Done!\nSave the model? y/[n]")
    ans = input('')
    if ans == 'y':
        model.save(os.path.join(work_path, "ANN_trained_" + marker, "model.keras"))
        f_extra = open(os.path.join(work_path, "ANN_trained_" + marker + "extras.csv"), 'w')
        for i in range(len(max_val)):
            f_extra.write(str(max_val[i]))
            if i != len(max_val) - 1: f_extra.write(', ')
        f_extra.close()
# End of __main__ section

In [ ]:
# Model caller
if __name__ == '__main__':
    print("Initialising...")
    #this_process = psutil.Process(os.getpid())
    #this_process.nice(psutil.BELOW_NORMAL_PRIORITY_CLASS)
    work_path = os.path.join("k:" + os.sep, "Archive", "Studying", "NeuralNets", "Python", "Jupyter - Phycocyanin ANN")
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus: tf.config.experimental.set_memory_growth(gpu, True)
    marker = 'C'
    low_priority_enabled = False
    if low_priority_enabled:
        import psutil
        this_process = psutil.Process(os.getpid())
        this_process.nice(psutil.BELOW_NORMAL_PRIORITY_CLASS)
    # Set RNG
    seed = 128
    rng = np.random.RandomState(seed)
    # Static parameters
    dataset_length = 0 # Test set, as many as we read
    train_list_length = 'F'  # The one I should use for batch creator - no longer a global var
    input_num_units = 12 #
    output_num_units = 3 # 3 is all
    #
    # Reading data
    df = pd.read_csv(os.path.join(work_path, "Test data C.csv"))
    df.info()
    print()
    df.drop(columns=['No', 'Init OD680', 'OD680'], inplace=True) # Drop the ID and 680s
    df = df.replace(to_replace='???', value=np.NaN) # Replace '???'s with true NaNs for ease of processing
    # Replace NaNs with means
    for i in df.columns[df.isnull().any(axis=0)]:
        df[i] = df[i].astype(np.float64) # Convert to float64 explicitly since there were string objects there
        df[i].fillna(df[i].mean(),inplace=True)
    # No adjustments should be required here other than loading and applying max values
    f_extra = open(os.path.join(work_path, "ANN_trained_" + marker + "extras.csv"), 'r')
    line_temp = ((f_extra.readline()).split(','))
    max_val = np.asarray(line_temp, dtype='float64')  # Max values have to be loaded for the trained module so we can un-normalize the results
    f_extra.close()
    max_val = list([x] for x in max_val) # Magic: we need max_val to be a dataframe.max() entity to divide another dataframe by it
    max_val = pd.DataFrame(np.transpose(max_val), columns = df.columns.to_list()) # Single-row dataframe with column names taken from the main data
    max_val = max_val[max_val.columns.to_list()].max() # ...and calling pandas max() on it
    for i in df.columns:
        df[i] = df[i].div(max_val[i]) # How do I even handle this now?..
    max_val = max_val.to_numpy()
    # Now, make it into a numpy array
    dataset = df.to_numpy()
    del df
    #print("Max values are:\n", np.around(max_val, decimals=5))
    #print("Mean values are:\n", np.around(mean_val, decimals=5))
    #print("Processed dataset example:\n", dataset[55])
    print("Dataset shape is:", dataset.shape)
    #
    dataset_length = len(dataset)
    model = load_model(os.path.join(work_path, "ANN_trained_" + marker, "model.keras"))
    print("Starting the process...")
    test_in_array = np.zeros((dataset_length, input_num_units))
    #test_list = np.zeros((valset_length, input_num_units + output_num_units))
    for i in range(dataset_length):
        next_line = dataset[i]
        test_in_array[i] = (next_line.copy())[:input_num_units]
    output_y = model(test_in_array, training=False)
    print("\nDone!")
    # Now, unnormalize the outputs
    y_pred = np.zeros((dataset_length, output_num_units))
    for i in range(output_num_units):
        for j in range(dataset_length):
            y_pred[j][i] = output_y[j][i]*max_val[i+input_num_units]
    print("Results:")
    y_pred = np.transpose(y_pred)
    for i in range(output_num_units):
        print((["OD_arr = ", "pH_arr = ", "mrel_arr = "])[i], end='[')
        for j in range(dataset_length):
            print(np.around(y_pred[i][j], decimals=5), end='')
            if j != dataset_length-1: print(end=', ')
        print(']')
    OD_arr = list(map(float, np.around(y_pred[0], decimals=5).tolist()))
    pH_arr =  list(map(float, np.around(y_pred[1], decimals=5).tolist()))
    mrel_arr =  list(map(float, np.around(y_pred[2], decimals=5).tolist()))
# End of __main__ section

<div class="alert alert-block alert-info"><b>Plotter for the prediction data (and an optional cell to load stats into it directly)</b></div>
Kernel crashes (or used to crash) if the plotter is ran after the model caller for some ungodly reason
Crash persists even if I override the variables using the optional load cell, which would point at a conflict with imported modules...
But it works fine if I just import everything from the init cell, so I have no idea what can be causing it

In [ ]:
if __name__ == '__main__':
    # Stat block example for testing:
    # 1
    OD_arr_1 = [1.09201, 1.0707, 1.04794, 1.02439, 1.00131, 0.98019, 0.9623, 0.94815, 0.93735, 0.92913, 0.92288, 0.91826, 0.91506, 0.91307, 0.912, 0.9116, 0.91162, 0.9119, 0.91233, 0.91282]
    pH_arr_1 = [10.47036, 10.48657, 10.50648, 10.52904, 10.55254, 10.57517, 10.59603, 10.6153, 10.63327, 10.64957, 10.66342, 10.67424, 10.68209, 10.68753, 10.69133, 10.69415, 10.69646, 10.6985, 10.7004, 10.70221]
    mrel_arr_1 = [208.2491, 203.79428, 198.45261, 192.3342, 185.66913, 178.74594, 171.81584, 165.00217, 158.29121, 151.66249, 145.21727, 139.17699, 133.79166, 129.24373, 125.59911, 122.81176, 120.76424, 119.31059, 118.30856, 117.63526]
    # 2
    OD_arr_2 = [0.85942, 0.75807, 0.72685, 0.75392, 0.80918, 0.85856, 0.89337, 0.9177, 0.93516, 0.9478, 0.95687, 0.9632, 0.96739, 0.96986, 0.97094, 0.9709, 0.96995, 0.96829, 0.96606, 0.9634]
    pH_arr_2 = [10.63946, 10.90057, 10.92727, 10.62086, 10.2704, 10.11689, 10.09033, 10.10995, 10.13969, 10.16716, 10.18904, 10.20504, 10.21571, 10.22173, 10.22378, 10.22243, 10.2182, 10.21153, 10.20282, 10.19242]
    mrel_arr_2 = [125.42416, 69.01167, 34.998, 6.4755, -0.24156, 11.18675, 22.00166, 29.06584, 33.48273, 36.29147, 38.11128, 39.29529, 40.05103, 40.50531, 40.7401, 40.81089, 40.75706, 40.6086, 40.38791, 40.11334]
    # 3
    OD_arr_3 = [0.54696, 0.61755, 0.68615, 0.74509, 0.79256, 0.83038, 0.86158, 0.88902, 0.9149, 0.94075, 0.96754, 0.99582, 1.02583, 1.05757, 1.09084, 1.12528, 1.1604, 1.19558, 1.23016, 1.26343]
    pH_arr_3 = [9.50936, 9.83099, 10.07716, 10.21466, 10.24782, 10.20252, 10.10986, 9.99695, 9.88375, 9.78326, 9.70299, 9.64653, 9.61486, 9.60731, 9.62211, 9.65679, 9.70837, 9.77357, 9.84891, 9.93089]
    mrel_arr_3 = [273.42175, 239.74292, 205.81673, 174.39099, 146.83078, 123.4855, 104.16939, 88.46951, 75.90604, 66.01046, 58.36932, 52.64514, 48.58485, 46.0163, 44.83905, 45.00928, 46.5232, 49.40006, 53.66481, 59.33297]
    # 4
    OD_arr_4 = [1.82769, 1.79618, 1.76355, 1.73063, 1.69785, 1.66499, 1.63109, 1.59464, 1.55393, 1.50756, 1.45484, 1.39602, 1.33224, 1.26547, 1.1983, 1.13372, 1.0749, 1.02484, 0.98612, 0.96061]
    pH_arr_4 = [10.88573, 10.89153, 10.89483, 10.89598, 10.89489, 10.89104, 10.88351, 10.87127, 10.85354, 10.83004, 10.80091, 10.76643, 10.72668, 10.68124, 10.62918, 10.56909, 10.49924, 10.41778, 10.32288, 10.21296]
    mrel_arr_4 = [-200.26862, -181.49881, -163.16299, -145.99054, -130.60416, -117.35041, -106.16136, -96.50732, -87.47782, -77.96059, -66.85262, -53.22567, -36.4099, -16.0043, 8.15626, 36.04226, 67.46478, 102.10805, 139.5477, 179.27197]
    # 5
    OD_arr_5 = [1.20154, 1.21214, 1.222, 1.23117, 1.23956, 1.24696, 1.25315, 1.25788, 1.26089, 1.26194, 1.26081, 1.25729, 1.25122, 1.24246, 1.23094, 1.21665, 1.19966, 1.18012, 1.15829, 1.13455]
    pH_arr_5 = [10.86854, 10.86799, 10.86735, 10.86706, 10.8669, 10.86632, 10.8646, 10.86104, 10.85495, 10.84579, 10.83313, 10.8167, 10.7964, 10.77233, 10.74481, 10.71433, 10.68158, 10.64742, 10.6128, 10.57871]
    mrel_arr_5 = [-44.05445, -55.80488, -64.13792, -69.57851, -72.57948, -73.51572, -72.68646, -70.32218, -66.5937, -61.62364, -55.49421, -48.25843, -39.94883, -30.58673, -20.18958, -8.77965, 3.60774, 16.91797, 31.07091, 45.95757]
    # 6 - with new RNG seed setup
    OD_arr_6 = [0.99085, 0.99686, 1.00305, 1.00995, 1.0182, 1.02833, 1.04065, 1.05497, 1.07044, 1.08548, 1.09789, 1.1052, 1.10505, 1.09571, 1.07664, 1.04878, 1.01466, 0.97783, 0.94203, 0.91023]
    pH_arr_6 = [10.72845, 10.70503, 10.68468, 10.66769, 10.65443, 10.64523, 10.64038, 10.6399, 10.64334, 10.64948, 10.65612, 10.66002, 10.65736, 10.64453, 10.61934, 10.58217, 10.53649, 10.48801, 10.44305, 10.40653]
    mrel_arr_6 = [126.44959, 142.05733, 156.45741, 169.39882, 180.6403, 189.97775, 197.27032, 202.45554, 205.54002, 206.56195, 205.53741, 202.41795, 197.09543, 189.47862, 179.63477, 167.94167, 155.15324, 142.29437, 130.39604, 120.21179]
    # 7
    OD_arr_7 = [0.99085, 0.99686, 1.00305, 1.00995, 1.0182, 1.02833, 1.04065, 1.05497, 1.07044, 1.08548, 1.09789, 1.1052, 1.10505, 1.09571, 1.07664, 1.04878, 1.01466, 0.97783, 0.94203, 0.91023]
    pH_arr_7 = [10.72845, 10.70503, 10.68468, 10.66769, 10.65443, 10.64523, 10.64038, 10.6399, 10.64334, 10.64948, 10.65612, 10.66002, 10.65736, 10.64453, 10.61934, 10.58217, 10.53649, 10.48801, 10.44305, 10.40653]
    mrel_arr_7 = [126.44959, 142.05733, 156.45741, 169.39882, 180.6403, 189.97775, 197.27032, 202.45554, 205.54002, 206.56195, 205.53741, 202.41795, 197.09543, 189.47862, 179.63477, 167.94167, 155.15324, 142.29437, 130.39604, 120.21179]
    # 8
    OD_arr_8 = [0.99085, 0.99686, 1.00305, 1.00995, 1.0182, 1.02833, 1.04065, 1.05497, 1.07044, 1.08548, 1.09789, 1.1052, 1.10505, 1.09571, 1.07664, 1.04878, 1.01466, 0.97783, 0.94203, 0.91023]
    pH_arr_8 = [10.72845, 10.70503, 10.68468, 10.66769, 10.65443, 10.64523, 10.64038, 10.6399, 10.64334, 10.64948, 10.65612, 10.66002, 10.65736, 10.64453, 10.61934, 10.58217, 10.53649, 10.48801, 10.44305, 10.40653]
    mrel_arr_8 = [126.44959, 142.05733, 156.45741, 169.39882, 180.6403, 189.97775, 197.27032, 202.45554, 205.54002, 206.56195, 205.53741, 202.41795, 197.09543, 189.47862, 179.63477, 167.94167, 155.15324, 142.29437, 130.39604, 120.21179]
    # old
    #OD_arr = [1.40069, 1.49881, 1.50349, 1.41423, 1.26278, 1.09557, 0.93667, 0.80344, 0.72084, 0.68163, 0.66574, 0.66062, 0.66031, 0.66207, 0.6646, 0.66742, 0.67043, 0.6737, 0.67736, 0.68151]
    #pH_arr = [10.84752, 10.84961, 10.79603, 10.69566, 10.57886, 10.45923, 10.34241, 10.25274, 10.1972, 10.16705, 10.15219, 10.1458, 10.14392, 10.14433, 10.14591, 10.14816, 10.15094, 10.15432, 10.15838, 10.16321]
    #mrel_arr = [314.75522, 309.40945, 293.15125, 262.86047, 221.04347, 175.806, 129.09119, 86.99325, 59.74993, 45.79125, 39.05718, 35.69876, 33.92191, 32.92041, 32.31872, 31.93623, 31.68605, 31.52694, 31.43983, 31.41485]
# End of __name__ section

In [ ]:
if __name__ == '__main__':
    import matplotlib.pyplot as plt
    x_list = list(range(5, 105, 5)) # NaNO3 concentration values - manual for now, mainly because it's a different cell and dataframe is unloaded
    print('OD\u2087\u2085\u2080 graph:')
    plt.figure(figsize=(16,10), dpi=100)
    #plt.plot(x_list, OD_arr_1, 'b.-', label='OD\u2087\u2085\u2080')
    #plt.plot(x_list, OD_arr_2, 'b.-')
    #plt.plot(x_list, OD_arr_3, 'b.-')
    #plt.plot(x_list, OD_arr_4, 'b.-')
    #plt.plot(x_list, OD_arr_5, 'b.-')
    plt.plot(x_list, OD_arr_6, 'b.-')
    plt.plot(x_list, OD_arr_7, 'b.-')
    plt.plot(x_list, OD_arr_8, 'b.-')
    #plt.title('Different NaNO\u2083 concentration: OD\u2087\u2085\u2080', fontsize=20)
    #plt.xlabel('NaNO\u2083 concentration', fontsize=16)
    #plt.ylabel('OD\u2087\u2085\u2080', fontsize=16)
    plt.title('Зависимость OD\u2087\u2085\u2080 от содержания соединений азота', fontsize=20)
    plt.xlabel('Содержание NaNO\u2083', fontsize=16)
    plt.ylabel('OD\u2087\u2085\u2080', fontsize=16)
    plt.legend()
    plt.grid()
    plt.show()
    print('pH graph:')
    plt.figure(figsize=(16,10), dpi=100)
    #plt.plot(x_list, pH_arr_1, 'r.-', label='pH')
    #plt.plot(x_list, pH_arr_2, 'r.-')
    #plt.plot(x_list, pH_arr_3, 'r.-')
    #plt.plot(x_list, pH_arr_4, 'r.-')
    #plt.plot(x_list, pH_arr_5, 'r.-')
    plt.plot(x_list, pH_arr_6, 'r.-')
    plt.plot(x_list, pH_arr_7, 'r.-')
    plt.plot(x_list, pH_arr_8, 'r.-')
    #plt.title('Different NaNO\u2083 concentration: OD\u2087\u2085\u2080', fontsize=20)
    #plt.xlabel('NaNO\u2083 concentration', fontsize=16)
    #plt.ylabel('pH', fontsize=16)
    plt.title('Зависимость pH среды от содержания соединений азота', fontsize=20)
    plt.xlabel('Содержание NaNO\u2083', fontsize=16)
    plt.ylabel('pH среды', fontsize=16)
    plt.legend()
    plt.grid()
    plt.show()
    print('mrel graph:')
    plt.figure(figsize=(16,10), dpi=100)
    #plt.plot(x_list, mrel_arr_1, 'g.-', label='Уд. содерж. фикоцианина')
    #plt.plot(x_list, mrel_arr_2, 'g.-')
    #plt.plot(x_list, mrel_arr_3, 'g.-')
    #plt.plot(x_list, mrel_arr_4, 'g.-')
    #plt.plot(x_list, mrel_arr_5, 'g.-')
    plt.plot(x_list, mrel_arr_6, 'g.-')
    plt.plot(x_list, mrel_arr_7, 'g.-')
    plt.plot(x_list, mrel_arr_8, 'g.-')
    #plt.title('Different NaNO\u2083 concentration: OD\u2087\u2085\u2080', fontsize=20)
    #plt.xlabel('NaNO\u2083 concentration', fontsize=16)
    #plt.ylabel('pH', fontsize=16)
    plt.title('Зависимость удельного содержания от содержания соединений азота', fontsize=20)
    plt.xlabel('Содержание NaNO\u2083', fontsize=16)
    plt.ylabel('Удельное содержание фикоцианина', fontsize=16)
    plt.legend()
    plt.grid()
    plt.show()
# End of __main__ section

<div class="alert alert-block alert-info"><b>Stand-alone version with per-epoch statistics.</b></div>
Can be used for running overfitting tests or for general testing of new configurations.

In [ ]:
import os

import pandas as pd
import numpy as np
import tensorflow as tf
#from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.models import load_model

from itertools import cycle
from tensorflow.keras import backend as K
from tqdm.keras import TqdmCallback

#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"   # Disable GPU if necessary

In [ ]:
def noisy_duplicate_keras_b(data_array_loc): # alt version, might fix the memory issue
    data_array_new = data_array_loc.copy()
    # Element-by-element
    for i in range(trainset_length):
        for j in range(input_num_units):
            data_array_new[i][j] = data_array_new[i][j]*(1 + noise_factor*(0.5 - np.random.random_sample()))
    return data_array_new

def hp_config_print(hp_list):
    label_list = ["Config: ", "/", "/", " - ", "/", "/", ", B ", ", E/LR ", "/", ", EF/NF ", "/"]
    for item, label in zip(hp_list, label_list): print(label, item, sep='', end='')
    print()

def trainset_generator(x_array, y_array, batch_size):
    pairs = [(x, y) for x in x_array for y in y_array]
    cycle_pairs = cycle(pairs)
    while (True):
        x_batch = []
        y_batch = []
        for _ in range(batch_size):
            x, y = next(cycle_pairs)
            x_batch.append(x)
            y_batch.append(y)
        yield np.array(x_batch), np.array(y_batch)

def met_R2(y_true, y_pred):
    SS_res = K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res/(SS_tot + K.epsilon()))

In [ ]:
# A separate version of the model trainer intented for single runs to test overfitting
if __name__ == '__main__':
    print("Initialising...")
    work_path = os.path.join("k:" + os.sep, "Archive", "Studying", "NeuralNets", "Python", "Jupyter - Phycocyanin ANN")
    generator_enabled = False
    low_priority_enabled = False
    pickle_enabled = False
    flag = 0
    if low_priority_enabled:
        import psutil
        this_process = psutil.Process(os.getpid())
        this_process.nice(psutil.BELOW_NORMAL_PRIORITY_CLASS)
    if pickle_enabled:
        import pickle
    # Set RNG
    seed = 128
    rng = np.random.RandomState(seed)
    tf.random.set_seed(seed) # Perhaps I need to set tf. seed separately?
    # Static parameters
    dataset_length = 490 # Out of 490 total
    train_list_length = dataset_length
    input_num_units = 12 #
    output_num_units = 3 # 3 is all
    #
    # Reading data
    df = pd.read_csv(os.path.join(work_path, "Exp data SE 7 (good-504).csv"))
    #df.info()
    df.drop(columns=['No', 'Init OD680', 'OD680'], inplace=True) # Drop the ID and 680s
    df = df.replace(to_replace='???', value=np.NaN) # Replace '???'s with true NaNs for ease of processing
    # Replace NaNs with means
    for i in df.columns[df.isnull().any(axis=0)]:
        df[i] = df[i].astype(np.float64) # Convert to float64 explicitly since there were string objects there
        df[i].fillna(df[i].mean(),inplace=True)
    mean_val = (df[df.columns.to_list()].mean()).to_numpy() # Not needed, testing only
    max_val = (df[df.columns.to_list()].max()) # Divide wont' work with a np_array of max values
    for i in df.columns:
        df[i] = df[i].div(max_val[i]) # Divide one by one because if not, the columns will get sorted alphabetically
    max_val = max_val.to_numpy()
    # Valset
    if 1==1: # Are we doing the manual selected test set to imitate the main program behavior?
        # Manually selected test dataset of 50 elements
        valset_length = 50 # 50
        test_points_list = [ 22,  28,  32,  41,  51,  55,  68,  73,  85,  91,
                             96,  97, 109, 119, 124, 137, 150, 166, 173, 181,
                            188, 193, 204, 214, 222, 241, 250, 259, 267, 280,
                            292, 311, 319, 333, 342, 350, 361, 372, 378, 385,
                            390, 394, 401, 408, 415, 435, 450, 461, 473, 482
                           ]
        # Issues in lines 1 and 4
        # In 300, 311, 319, 333, 342
        # 300 causes decent improvement
        # 311 not much, 319 not much, 333 not much, 342 half-decent
        # Replacements for 300:
        # 290: meh 291: good 292: good+ 293: good+ 294: good+ 295: good 296: good+ 297: meh 298: good+ 299: good
        test_dataset = (df.loc[test_points_list]).to_numpy()
        df.drop(test_points_list, axis=0, inplace=True)
        train_list_length = 440
    # Now, make it into a numpy array
    dataset = df.to_numpy()
    del df
    #print("Max values are:\n", np.around(max_val, decimals=5))
    #print("Mean values are:\n", np.around(mean_val, decimals=5))
    #print("Processed dataset example:\n", dataset[55])
    print("Dataset shape is:", dataset.shape)
    #
    # Hyperparameters
    hyperparameters = [10, 8, 9, 'tanh', 'tanh', 'sigmoid', 40, 5000, 0.001, 4, 0.05]
    print("Starting the process...")
    #
    hidden1_num_units, hidden2_num_units, hidden3_num_units, hidden1_func, hidden2_func, hidden3_func, batch_size, epochs, learning_rate, extension_factor, noise_factor = hyperparameters
    hp_config_print(hyperparameters)
    # Data normalization for the noise
    dataset_norm = (dataset.copy())/(1 + noise_factor)  # Legacy
    # Keras: creating a model for the set of hyperparameters to be used for a single stat loop
    # The model is then cleared by clear_session
    model = Sequential([
        Dense(units=hidden1_num_units, input_shape=(input_num_units,), activation=hidden1_func),
        Dense(units=hidden2_num_units, activation=hidden2_func),
        Dense(units=hidden3_num_units, activation=hidden3_func),
        Dense(units=output_num_units, activation='linear')
    ])
    #model.summary() # Debug
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=[met_R2])
    # Randomising data selection: grab the whole set and shuffle it
    # I can't use Keras shuffle and validation split because of the inflation algorithm
    # This way test set is different every time and randomly selected while still being X pre-selected numbers
    trainset_length = train_list_length
    train_in_array = np.zeros((trainset_length, input_num_units))
    train_out_array = np.zeros((trainset_length, output_num_units))
    #
    val_in_array = np.zeros((valset_length, input_num_units))
    val_out_array = np.zeros((valset_length, output_num_units))
    #
    #rand_list = np.random.choice(trainset_length, trainset_length, False)
    for i in range(trainset_length):
        line_to_add = dataset_norm[i]
        train_in_array[i] = (line_to_add[:input_num_units].copy())/(1 + noise_factor)
        train_out_array[i] = (line_to_add[input_num_units:].copy())
    for i in range(valset_length):
        line_to_add = test_dataset[i]
        val_in_array[i] = (line_to_add[:input_num_units].copy())/(1 + noise_factor)
        val_out_array[i] = (line_to_add[input_num_units:].copy())
    # Extending dataset with 'noisy' duplicates
    train_in_array_temp = train_in_array.copy()
    train_out_array_temp = train_out_array.copy()
    for i in range(0, extension_factor):
        train_in_array_temp = np.concatenate((train_in_array_temp, noisy_duplicate_keras_b(train_in_array)))  # Noisy inputs
        train_out_array_temp = np.concatenate((train_out_array_temp, train_out_array))                      # Clean corresponding outputs
    train_in_array = train_in_array_temp.copy()    # Copy because I'm clearing it, which is probably unnecessary
    train_out_array = train_out_array_temp.copy()
    del train_in_array_temp
    del train_out_array_temp
    #
    train_list_length = train_list_length*(1 + extension_factor)   # Legacy
    if len(train_in_array) != train_list_length: print("--- Train list length is off! ---")
    # train_list = dataset[:dataset_length]
    # test_list = dataset[dataset_length:]
    #print("Train input shape is:", train_in_array.shape, "train output shape is:", train_out_array.shape)
    #print("Test input shape is:", test_in_array.shape, "test output shape is:", test_out_array.shape)
    # Calling the Keras model to train
    # Fit: suffle=False because it is already shuffled because of how the dataset is split between train and test
    #print("Doing fit...")
    if generator_enabled:
        trainset = trainset_generator(train_in_array, train_out_array, batch_size)
        #model.fit(trainset, steps_per_epoch=len(train_in_array)/batch_size, batch_size=batch_size, epochs=epochs, shuffle=False, verbose=0, callbacks=[])
        fit_history = model.fit(
            trainset, validation_data=(val_in_array, val_out_array), steps_per_epoch=len(train_in_array)/batch_size, 
            batch_size=batch_size, epochs=epochs, shuffle=False, verbose=0, callbacks=[TqdmCallback(verbose=0)]
        )
    else:
        fit_history = model.fit(
            x=train_in_array, y=train_out_array, validation_data=(val_in_array, val_out_array), 
            batch_size=batch_size, epochs=epochs, shuffle=False, verbose=0, callbacks=[TqdmCallback(verbose=0)]
        )
    print("Done!", end=' ')
    eval_results = model.evaluate(val_in_array, val_out_array)
    print("Loss, metrics:", eval_results)
    # No need to save the model here
    # Now, grab the history to plot a learning curve
    history_dict = fit_history.history
    if pickle_enabled: # Outdated: I have to use pickle dump because the core dies if I try to plot with everythign else loaded
        f = open(os.path.join(work_path, "History_dump_temp", 'wb'))
        pickle.dump(history_dict, f, 2)
        f.close
        f = open(os.path.join(work_path, "Epochs_dump_temp", 'wb'))
        pickle.dump(epochs, f, 2)
        f.close
        print("Dump done!")
# End of __main__ section

In [ ]:
# Plotter for the learning curves. Pray to the Omnissaiah that it doesn't crash between these two cells
if __name__ == '__main__':
    import matplotlib.pyplot as plt
    if not 'pickle_enabled' in locals(): import pickle # If the var is not defined, the previous cell didn't run in this kernel, and as such we need to load from pickle
    if pickle_enabled: 
        f = open(os.path.join(work_path, "History_dump_temp", 'rb'))
        history_dict = pickle.load(f)
        f.close
        f = open(os.path.join(work_path, "Epochs_dump_temp", 'rb'))
        epochs = pickle.load(f)
        f.close
    stat_loss_fit = history_dict['loss']
    stat_loss_val = history_dict['val_loss']
    stat_acc_fit = history_dict['met_R2']
    stat_acc_val = history_dict['val_met_R2']
    x_list = list(range(0, epochs))
    print('MSE graph:')
    plt.figure(figsize=(16,10), dpi=100)
    plt.plot(x_list, stat_loss_fit, 'b-', label='Обучающие данные')
    plt.plot(x_list, stat_loss_val, 'g-', label='Валидационные данные')
    plt.title('Зависимость среднеквадратичной ошибки от количества эпох при обучении', fontsize=20)
    plt.xlabel('Эпохи', fontsize=16)
    plt.ylabel('Среднеквадратичная ошибка', fontsize=16)
    plt.legend()
    plt.grid()
    plt.ylim([0.0, 0.03])
    plt.show()
    print('R2 graph:')
    plt.figure(figsize=(16,10), dpi=100)
    plt.plot(x_list, stat_acc_fit, 'b-', label='Обучающие данные')
    plt.plot(x_list, stat_acc_val, 'g-', label='Валидационные данные')
    plt.title('Зависимость коэффициента детерминации от количества эпох при обучении', fontsize=20)
    plt.xlabel('Эпохи', fontsize=16)
    plt.ylabel('Коэффициент детерминации', fontsize=16)
    plt.legend()
    plt.grid()
    plt.ylim([0.5, 1.0])
    plt.show()
# End of __main__ section

<div class="alert alert-block alert-info"><b>Recalculations of phycocyanin dry weight equations</b></div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
od_arr_1 = np.asarray([0.04, 0.039, 0.071, 0.072, 0.071, 0.072, 0.078, 0.068, 0.053, 0.038,
                       0.033, 0.03, 0.156, 0.149, 0.141, 0.146, 0.138, 0.131, 0.075, 0.056,
                       0.057, 0.062, 0.332, 0.335, 0.32, 0.28, 0.314, 0.306, 0.08, 0.082,
                       0.51, 0.505, 1.22, 1.171, 1.156, 1.148, 1.175, 1.024, 0.106, 0.091,	
                       0.445, 0.512, 1.251, 1.265, 1.221, 1.193, 1.273, 1.15, 0.923, 0.905,
                       0.872, 0.875, 0.874, 0.912, 0.827, 0.841, 0.191, 0.182, 1.041, 0.998, 
                       0.962, 0.984, 0.956, 1.003, 0.981, 0.977, 0.407, 0.368, 0.401, 0.351,
                       0.448, 	0.376, 	0.437, 	0.392, 	0.482, 0.479])
dw_arr_1 = np.asarray([0.002611097163, 0.002611097163, 0.00203504805, 0.00203504805, 0.00306591722, 0.00306591722, 0.005983545251, 0.005983545251, 0.00219890055, 0.00219890055,
                       -0.0001672491263, -0.0001602596106, 0.0001763762694, 0.0001630144308, 0.0002486569455, 0.0002601688411, 0.00008840353614, 0.00008138325533, 0.00008556149733, 0.00008225108225,
                       0.00007581047382, 0.00008952618454, 0.0003256924546, 0.0003190066858, 0.000247244489, 0.000253256513, 0.000342203432, 0.0003312608804, 0.0001570680628, 0.0001775118424,
                       0.001596806387, 0.001596806387, 0.007412972702, 0.007412972702, 0.004806007509, 0.004806007509, 0.004189526185, 0.004189526185, 0.0003967270022, 0.0003967270022,
                       0.003066700741, 0.003066700741, 0.009765819631, 0.009765819631, 0.006745017885, 0.006745017885, 0.007607298895, 0.007607298895, 0.002611097163, 0.002611097163,
                       0.00203504805, 0.00203504805, 0.00306591722, 0.00306591722, 0.005983545251, 0.005983545251, 0.00219890055, 0.00219890055, 0.003329161452, 0.003329161452,
                       0.002813363477, 0.002813363477, 0.002882882883, 0.002882882883, 0.004952830189, 0.004952830189, 0.004954954955, 0.005555555556, 0.004954954955, 0.005555555556,
                       0.004954954955, 0.005555555556, 0.004954954955, 0.005555555556, 0.004954954955, 0.005555555556])*1000
i_1 = np.argsort(od_arr_1)
od_arr_1 = od_arr_1[i_1]
dw_arr_1 = dw_arr_1[i_1]
poly_1 = np.polynomial.polynomial.Polynomial.fit(od_arr_1, dw_arr_1, 1).convert().coef
x_1 = od_arr_1[:,np.newaxis]
a_1, _, _, _ = np.linalg.lstsq(x_1, dw_arr_1, rcond=None)
print(poly_1, a_1)
plt.figure()
plt.plot(od_arr_1, dw_arr_1, 'b.')
plt.plot((min(od_arr_1), max(od_arr_1)), np.polynomial.polynomial.polyval((min(od_arr_1), max(od_arr_1)), poly_1, False), 'b-')
plt.plot(x_1, a_1*x_1, 'g-')
plt.grid()
plt.show()
#
od_arr_2 = np.asarray([0.709, 0.369, 0.201, 0.096, 0.042, 0.019])
dw_arr_2 = np.asarray([0.0003368421053, 0.0001947368421, 0.0001184210526, 0.00006842105263, 0.00003421052632, 0.00002894736842])*1000
i_2 = np.argsort(od_arr_2)
od_arr_2 = od_arr_2[i_2]
dw_arr_2 = dw_arr_2[i_2]
poly_2 = np.polynomial.polynomial.Polynomial.fit(od_arr_2, dw_arr_2, 1).convert().coef
x_2 = od_arr_2[:,np.newaxis]
a_2, _, _, _ = np.linalg.lstsq(x_2, dw_arr_2, rcond=None)
print(poly_2, a_2)
plt.figure()
plt.plot(od_arr_2, dw_arr_2, 'b.')
plt.plot((min(od_arr_2), min(od_arr_2)), np.polynomial.polynomial.polyval((min(od_arr_2), min(od_arr_2)), poly_2, False), 'g-')
plt.plot(x_2, a_2*x_2, 'g-')
plt.grid()
plt.show()